### IS - Brand position
Finding Product position on Semantic Network (PPSN)

similarity: 1 คือเหมือน 0 คือต่าง<br>
dissimilarity 1 คือต่าง 0 คือเหมือน : 

In [1]:
import networkx as nx
import numpy as np
import pandas as pd
import itertools
import math
import string
import json
import pymongo
from itertools import groupby

In [2]:
import gensim
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=pd.core.common.SettingWithCopyWarning)

C:\ProgramData\Anaconda3\lib\site-packages\past\types\oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable
C:\ProgramData\Anaconda3\lib\site-packages\past\builtins\misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping


In [3]:
import time
from datetime import datetime
today = datetime.now()

In [4]:
# PyThaiNLP v2.3.0-beta1 is The first beta release of PyThaiNLP 2.3

from pythainlp.util import normalize
#from newnewthaicut import word_tokenize  # IOB Tagging tokenized
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus import thai_stopwords
from pythainlp.spell import correct

from pythainlp.util import dict_trie
from pythainlp.corpus.common import thai_words

In [5]:
import statistics as stat
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.spatial.distance import cosine
#from sklearn.metrics.pairwise import cosine_similarity
#from sklearn.metrics import jaccard_score
#from mlxtend.evaluate import lift_score
#from gensim.models import TfidfModel

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [6]:
# Files definition
root_input_path = 'Datasource'
root_output_path = 'output'

comment_nlp_file = root_output_path+"/comment_nlpToken.csv"
comment_cooc_freq_file = root_output_path+"/comment_cooc_freq.xlsx"
comment_cooc_jacc_file = root_output_path+"/comment_cooc_jacc.xlsx"
comment_cooc_assoc_file = root_output_path+"/comment_cooc_assoc.xlsx"

comment_robust_jaccard = root_output_path+"/comment_robust_jaccard.xlsx"
comment_robust_lift = root_output_path+"/comment_robust_lift.xlsx"
comment_robust_incur = root_output_path+"/comment_robust_incur.xlsx"
comment_robust_cosine = root_output_path+"/comment_robust_cosine.xlsx"
comment_robust_assoc = root_output_path+"/comment_robust_assoc.xlsx"

In [7]:
# Connect MongoDB
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["NIDA_PPSN_PRD"]
col_thread = mydb["NIDA_PPSN_THREAD"]
col_comment = mydb["NIDA_PPSN_COMMENT"]
col_scrape = mydb["NIDA_PPSN_SCRAPE"]
col_claen = mydb["NIDA_PPSN_CLEANED_DATA"]

### 1.Clean data & NLP Tokenized

In [8]:
cursor_corpus = col_scrape.find()
df_Corpus = pd.DataFrame(cursor_corpus)

In [9]:
df_Corpus.head()

,_id,URLs,headline,text,article_date,Retrived_date
0,6058a2fe8fb21a3a76d2d5de,https://pantip.com/topic/39868603,นมเมจิรสหวานทำไมหายากจังคะ,เมื่อก่อนหลายปีมาแล้ว ในเซเว่นยังมีนมเมจิรสหวา...,2020-05-05 13:22:59,2021-03-22 21:00:24.737
1,6058a2fe8fb21a3a76d2d5df,https://pantip.com/topic/39868603,นมเมจิรสหวานทำไมหายากจังคะ,เบาหวานครับ แฮ่ๆ,2020-05-05 13:22:59,2021-03-22 21:00:24.737
2,6058a2fe8fb21a3a76d2d5e0,https://pantip.com/topic/39868603,นมเมจิรสหวานทำไมหายากจังคะ,หาซื้อไม่ได้นี่คิดได้ 2 อย่างนะ. คนไม่ค่อย...,2020-05-05 13:22:59,2021-03-22 21:00:24.737
3,6058a2fe8fb21a3a76d2d5e1,https://pantip.com/topic/39868603,นมเมจิรสหวานทำไมหายากจังคะ,เป็นรสที่ขายไม่ดีครับ\nรสพื้นฐานคือ\nจืด แบ่งย...,2020-05-05 13:22:59,2021-03-22 21:00:24.737
4,6058a2fe8fb21a3a76d2d5e2,https://pantip.com/topic/39868603,นมเมจิรสหวานทำไมหายากจังคะ,ถ้าขายไม่ดีเค้าก็เอาออกครับ shelf life มันน้อย...,2020-05-05 13:22:59,2021-03-22 21:00:24.737


In [10]:
print('All comment:',df_Corpus.shape[0])
print('All URLs ก่อนตัด:',len(df_Corpus.URLs.unique()))

All comment: 225360
All URLs ก่อนตัด: 4995


In [11]:
missing = df_Corpus.isnull().sum()
print(missing)

_id              0
URLs             0
headline         0
text             0
article_date     0
Retrived_date    0
dtype: int64


In [12]:
df_Corpus.tail()

,_id,URLs,headline,text,article_date,Retrived_date
225355,605c03f52f6c7bb657e87ed0,https://pantip.com/topic/30062751,ใครที่คิดจะไปออกกำลังกายที่ฟิตเนส มีคำแนะนำครับ,สรุปว่าควรเวทก่อนแล้วค่อย Cardio ใช่มั้ยคะ,2013-10-09 11:22:34,2021-03-25 10:30:08.736
225356,605c03f52f6c7bb657e87ed1,https://pantip.com/topic/30062751,ใครที่คิดจะไปออกกำลังกายที่ฟิตเนส มีคำแนะนำครับ,กระทู้นี้ไม่เห็นด้วยเรื่ิองเวย์ ที่บอกว่าอันตร...,2013-10-09 11:22:34,2021-03-25 10:30:08.736
225357,605c03f52f6c7bb657e87ed2,https://pantip.com/topic/30062751,ใครที่คิดจะไปออกกำลังกายที่ฟิตเนส มีคำแนะนำครับ,กระทู้นี้ไม่เห็นด้วยเรื่ิองเวย์ ที่บอกว่าอันตร...,2013-10-09 11:22:34,2021-03-25 10:30:08.736
225358,605c03f52f6c7bb657e87ed3,https://pantip.com/topic/30062751,ใครที่คิดจะไปออกกำลังกายที่ฟิตเนส มีคำแนะนำครับ,^^,2013-10-09 11:22:34,2021-03-25 10:30:08.736
225359,605c03f52f6c7bb657e87ed4,https://pantip.com/topic/30062751,ใครที่คิดจะไปออกกำลังกายที่ฟิตเนส มีคำแนะนำครับ,ดีครับ ขอบคุณ จขกท และสมาชิกอื่นๆที่มาช่วยแชร์...,2013-10-09 11:22:34,2021-03-25 10:30:08.736


In [13]:
df_Corpus.drop('_id',axis=1,inplace=True)

In [14]:
seqNum = list(range(1,df_Corpus.shape[0]+1))
df_Corpus.insert(0,'commentId',seqNum)
df_Corpus.columns = ['commentId','URLs','headline','text','article_date','Retrived_date']

In [15]:
df_Corpus.tail()

,commentId,URLs,headline,text,article_date,Retrived_date
225355,225356,https://pantip.com/topic/30062751,ใครที่คิดจะไปออกกำลังกายที่ฟิตเนส มีคำแนะนำครับ,สรุปว่าควรเวทก่อนแล้วค่อย Cardio ใช่มั้ยคะ,2013-10-09 11:22:34,2021-03-25 10:30:08.736
225356,225357,https://pantip.com/topic/30062751,ใครที่คิดจะไปออกกำลังกายที่ฟิตเนส มีคำแนะนำครับ,กระทู้นี้ไม่เห็นด้วยเรื่ิองเวย์ ที่บอกว่าอันตร...,2013-10-09 11:22:34,2021-03-25 10:30:08.736
225357,225358,https://pantip.com/topic/30062751,ใครที่คิดจะไปออกกำลังกายที่ฟิตเนส มีคำแนะนำครับ,กระทู้นี้ไม่เห็นด้วยเรื่ิองเวย์ ที่บอกว่าอันตร...,2013-10-09 11:22:34,2021-03-25 10:30:08.736
225358,225359,https://pantip.com/topic/30062751,ใครที่คิดจะไปออกกำลังกายที่ฟิตเนส มีคำแนะนำครับ,^^,2013-10-09 11:22:34,2021-03-25 10:30:08.736
225359,225360,https://pantip.com/topic/30062751,ใครที่คิดจะไปออกกำลังกายที่ฟิตเนส มีคำแนะนำครับ,ดีครับ ขอบคุณ จขกท และสมาชิกอื่นๆที่มาช่วยแชร์...,2013-10-09 11:22:34,2021-03-25 10:30:08.736


In [16]:
# Drop NA & banned comments
df_Corpus = df_Corpus.dropna()
df_Corpus = df_Corpus[df_Corpus["text"].str.find('ความคิดเห็นนี้ถูกลบ')!= 0]
df_Corpus = df_Corpus[df_Corpus["text"].str.find('แก้ไขข้อความเมื่อ')!= 0]
df_Corpus = df_Corpus.set_index('commentId')

In [17]:
missing = df_Corpus.isnull().sum()
print(missing)

URLs             0
headline         0
text             0
article_date     0
Retrived_date    0
dtype: int64


In [18]:
print('จำนวน comment หลังตัด:',df_Corpus.shape[0])
print('จำนวนกระทู้หลังตัด:',len(df_Corpus.URLs.unique()))

จำนวน comment หลังตัด: 221841
จำนวนกระทู้หลังตัด: 4994


In [19]:
df_Corpus.insert(4,'token_text',None)
df_Corpus.tail()

,URLs,headline,text,article_date,token_text,Retrived_date
commentId,,,,,,
225356,https://pantip.com/topic/30062751,ใครที่คิดจะไปออกกำลังกายที่ฟิตเนส มีคำแนะนำครับ,สรุปว่าควรเวทก่อนแล้วค่อย Cardio ใช่มั้ยคะ,2013-10-09 11:22:34,None,2021-03-25 10:30:08.736
225357,https://pantip.com/topic/30062751,ใครที่คิดจะไปออกกำลังกายที่ฟิตเนส มีคำแนะนำครับ,กระทู้นี้ไม่เห็นด้วยเรื่ิองเวย์ ที่บอกว่าอันตร...,2013-10-09 11:22:34,None,2021-03-25 10:30:08.736
225358,https://pantip.com/topic/30062751,ใครที่คิดจะไปออกกำลังกายที่ฟิตเนส มีคำแนะนำครับ,กระทู้นี้ไม่เห็นด้วยเรื่ิองเวย์ ที่บอกว่าอันตร...,2013-10-09 11:22:34,None,2021-03-25 10:30:08.736
225359,https://pantip.com/topic/30062751,ใครที่คิดจะไปออกกำลังกายที่ฟิตเนส มีคำแนะนำครับ,^^,2013-10-09 11:22:34,None,2021-03-25 10:30:08.736
225360,https://pantip.com/topic/30062751,ใครที่คิดจะไปออกกำลังกายที่ฟิตเนส มีคำแนะนำครับ,ดีครับ ขอบคุณ จขกท และสมาชิกอื่นๆที่มาช่วยแชร์...,2013-10-09 11:22:34,None,2021-03-25 10:30:08.736


เช็ควันเวลา

In [20]:
print('กระทู้เก่าที่สุด :',min(df_Corpus.article_date))
print('กระทู้ใหม่ที่สุด :',max(df_Corpus.article_date))

กระทู้เก่าที่สุด : 2013-01-03 09:20:18
กระทู้ใหม่ที่สุด : 2021-03-20 16:56:47


เก็บเฉพาะหัวกระทู้

In [21]:
df_thread = df_Corpus.copy()
df_thread = df_thread[['URLs','headline']]
df_thread.drop_duplicates(subset ="URLs", keep = 'first', inplace = True)
df_thread.insert(2,'token_headline',None)
df_thread.insert(3,'mention_product',None)
df_thread.insert(4,'mention_brand',None)
df_thread.tail()

,URLs,headline,token_headline,mention_product,mention_brand
commentId,,,,,
225076,https://pantip.com/topic/30038478,แนะนำด้วยคะ ไปฮ่องกงกับลูกสาวอายุขวบสี่เดือน,None,None,None
225080,https://pantip.com/topic/30149550,นมกล่อง uht เจาะแล้วอยู่ได้นานแค่ไหน และนมเปรี...,None,None,None
225085,https://pantip.com/topic/30023475,นมถุง ที่ขายในโรงเรียน มีขายที่อื่นในกทม.อีกไห...,None,None,None
225120,https://pantip.com/topic/30022399,ถามหาที่ขายนมแพะค่ะ,None,None,None
225126,https://pantip.com/topic/30062751,ใครที่คิดจะไปออกกำลังกายที่ฟิตเนส มีคำแนะนำครับ,None,None,None


## 2.ตัดคำและเก็บลง MongoDB (ใช้ insight จาก jupyter งานก่อน)

https://www.thainlp.org/pythainlp/tutorials/notebooks/pythainlp-get-started.html#Thai-Characters

In [22]:
def normThai(w):
    returnList = []
    for i in w:
        returnList.append(normalize(i))
    return returnList

In [23]:
custom_list = [#brand'
                'ดัชมิลล์','ดัชมิล','ดัชมิว','dutch mill','dutch milk','ดัชชี่','เมจิ','ซีพีเมจิ','ซีพี เมจิ','cp meiji','betagen','บีทาเก้น','บีทาเกน'
               ,'โฟร์โมสต์','โฟร์โมส','โฟโมสต์','โฟโมต','โฟโมส','โฟโม้ส','โฟรโมสต์','โฟรโมสต','โชคชัย','แดรี่โฮม','dairy home','เดรี่โฮม','เดลี่โฮม'
               ,'อืมม มิลค์','umm milk','เอ็มมิลค์','m milk','mmilk','เอ็มมิ้ลค์','เอ็มมิลล์','แมคโนเลีย','นมฟาร์มโชคชัย','ไทยเดนมาร์ค','ไทยเดนมาร์ก','นมวัวแดง','หนองโพ','คาเนชั่น'
               #flavor
               ,'สตอเบอร์รี่','สตรอว์เบอร์รี','รสสตรอเบอรี่','สตรอเบอรี่','สตรอวเบอรี่','สตอเบอรี่','สตรอเบอรี่','สตรอเบอร์รี่','สตรอเบอรี'
               ,'ชอคโกแล็ต','ช็อกโกแลต','ช็อคโกแลต','ช๊อกโกแลต','ช็อคโกแล็ต','ช็อกโกเลต','มิ้นท์ชอค','มิ้นต์ช้อก','รสกล้วย'
               ,'รสกาแฟ','รสหวาน','รสจืด','ไขมันต่ำ','ไขมัน 0%','low fat','0% fat','ขาดมันเนย','พร่องมันเนย','ไฮแคลเซียม'
               ,'high protein','whey formula','วานิลลา','วิปครีม','สูตรลดน้ำตาล'
               ,'ไฮโปรตีน','เวย์โปรตีน','whey','เวย์','อัลมอนด์','ซีเล็ค','ซีเล็คท์','กล้วยหอม','ไวท์คอฟฟี่','คาปูชิโน่','อเมริกาโน่'
               ,'grass fed','free lactose','lactose free','พาสเจอไรซ์','พาสเจอร์ไรซ์','พาสเจอร์ไรส์','พาซเจอไรซ์','พาสเจอร์ไลท์'
               ,'เมจิโกลด์ แม็กซ์','เมจิโกลด์','เมจิ โกลด์','เมจิโกล์ดแม็กซ์','เมจิ บัลแกเรีย','เมจิบัลแกเรีย','นมเปรี้ยว','รสธรรมชาติ','รสกลมกล่อม','ไพเกน'
               ,'โยเกิต','โยเกิรต','โยเกิร์ตพร้อมดื่ม','น้ำตาลมะพร้าว','นมฮอกไกโด','เบดไทม์','ฟรีแลคโตส','แลคโตสฟรี','ริชเอสเพสโซ่'
               ,'bedtime','bed time','เบดไทม์','ดาร์คช็อกโกแลต','ดาร์กช็อก','ดาร์กช็อกโกแลต','แอลคานีทีน'
               ,'คาราเมล','ช็อกโกมอลต์','เมล่อน','ชาเขียวมัจฉะ','ชาไทย','วิปครีม','whipping cream','nondairy','non dairy','Non dairy whipping cream'
               # ห้างร้าน
               ,'7 eleven','เซเว่น อีเลฟเว่น','เซเว่นอีเลฟเว่น','เซเว่น','เซเวน','เซ่เว่น','7 11','สตาร์บัค','อเมซอน'
               ,'ท็อปส์','ทอปส์','ท้อปส์','ท๊อปส์','แมคโคร','แม็คโคร','โลตัส','บิ๊กซี','bigc','golden place','big c','พนักงานขายนม'
               # อื่นๆ
               ,'ฝาน้ำเงิน','ผลิตภันท์นม','เต้าฮวยนมสด','ออแกนิค','ออแกนิก'
               ,'ขายไม่ดี','แพคคู่','ค่าจัดส่ง','shelf life','พนักงานขายนม','ซื้อประจำ','หายาก','หาซื้อ','หาซื้อไม่ได้','ของแถม','ราคาสูง','น้ำนมโค','นมโค','นมโคแท้','นมแพะ'
               ,'นมโรงเรียน','แพ้นม','แพ้นมวัว','นมอัดเม็ด','นมวัว','ตัวเตี้ย','ตัวสูง','แจกของ','ราคาถูก','ราคาแพง','นมแม่','ครีมเทียม','นมข้นหวาน','นมขวด'
               ,'เล่นเวท','นำ้หนัก','คุณแม่มือใหม่','นมอุ่น','ชานม','กินนม','ดื่มนม','ท้องเสีย','ขี้แตก','คุมอาหาร','นักวิ่ง','ร้านนมสด','ดูแลสุขภาพ','คนท้อง','มวลกระดูก'
               ,'คีเฟอร์นม','พันทิป','ร้านนม','เหมียวน้อย','ลูกสุนัข','ลูกหมา','คายทิ้ง','เจมส์ จิ','เจมส์จิ','ณเดช','ณเดชน์','สตอรี่' ,'อยากสูง','ส่วนสูง','สูงขึ้น','รักษามะเร็ง'
               ,'รักษาเบาหวาน','กระดูกพัง','กระดูกหัก','กระดูกแตก','กระดูกแข็งแรง','กระดูกยาว','กระดูกพรุน','ไม่ชอบ','ไม่ได้','ไม่อร่อย','ชาไข่มุก','ชานมไข่มุก'
               ,'นมข้น','อเมซอน','นมเมจิสีฟ้า','ทำฟอง','ตีฟอง','โฟมนม','มื้อเช้า','ไขมันทรานส์','ดาราเดลี่','แดรี่ฟาร์ม','แดรี่ควีน','ควบคุมอาหาร','เสริมสร้างร่างกาย'
               ,'ซ่อมแซมส่วนที่สึกหรอ','เสริมสร้างกระดูก','แอนตี้ไบโอติก','แพ็ก','แพกเกจ','แพ็กเกจ','แพ็ค','แพ๊ค','แพคเกจ','แพ็คเกจ','แคลเซี่ยม','เพิ่มน้ำหนัก'
               ,'ฟองนม','ไม่คุ้มค่า','ราคาที่จ่าย']
words = set(thai_words()).union(set(custom_list))
trie = dict_trie(dict_source=words)

In [24]:
garbage_char = ['ไม่','','.','..','...','มกราคม', 'กุมภาพันธ์','มีนาคม', 'เมษายน','พฤษภาคม', 'มิถุนายน', 'กรกฎาคม','สิงหาคม','กันยายน'
                ,'ตุลาคม','พฤศจิกายน', 'ธันวาคม','วันจันทร์','วันอังคาร','วันพุธ','วันพฤหัสบดี','วันศุกร์','วันเสาร์','วันอาทิตย์','กก'
                ,'เมนู','Menu','Net','net','สาขา','บาท','ราคา','ฯ','ๆ','กก','อันนี้','😆', '🤣','😢','😏','😂','😿','🥺','ววว','xx','อิอิ','แย้ววว']
naka = ['นะคะ','นะค่ะ','น่ะค่ะ','น่ะคะ','ฮ้าฟ','ค้าบ','คร้าบ']

stopwords = set(thai_stopwords()).union(set(naka))
stopwords.remove('ไม่')
stopwords.remove('สูง')

In [25]:
print('จำนวน comment หลังตัด:',df_Corpus.shape[0])
print('จำนวนกระทู้หลังตัด:',len(df_Corpus.URLs.unique()))

จำนวน comment หลังตัด: 221841
จำนวนกระทู้หลังตัด: 4994


In [26]:
def tokenize_custom(x):
    custom_punc = '!.—-"$#&฿/&\'()*+,:;<=>?@[\\]^_`{|}~'
    x = x.translate(str.maketrans('', '', custom_punc)).strip()
    x = x.translate(str.maketrans({"\t":None,"\n": None})).strip()
    x = x.lower()
    wtkn = word_tokenize(x, custom_dict=trie, engine='newmm')
    #wtkn = word_tokenize(x, engine='nercut') # if using custom dict, modify named_entity.py
    wtkn = [j for j in wtkn if j not in stopwords]
    wtkn = [w for w in wtkn if len(w) >= 2]
    wtkn = [y for y in wtkn if y not in garbage_char]
    wtkn = [s for s in wtkn if not s.isdigit()]
    wtkn = normThai(wtkn)
    return wtkn

In [27]:
#tokenize_custom('มานีมีลูกแมวดัชมิวเหมียวจริงๆ')

In [28]:
def create_InsertMany_Thread(df):
    listofdict = []
    for c,lx in enumerate(df.URLs,0):
        info = {
            "URLs": lx,
            "headline": df.headline[c],
            "token_headline":df.token_headline[c],
            "t_mention_dairy":df.t_mention_dairy[c],
            "t_mention_product": df.t_mention_product[c],
            "t_mention_brand": df.t_mention_brand[c]
        }
        listofdict.append(info)
    return listofdict

In [29]:
def create_InsertMany_comments(df):
    listofdict = []
    for c,lx in enumerate(df.URLs,0):
        info = {
            "commentId":int(df.commentId[c]),
            "URLs": lx,
            "headline":df.headline[c],
            "text": df.text[c],
            "Retrived_date": df.Retrived_date[c],
            "token_text":df.token_text[c],
            "cmt_mention_dairy": df.cmt_mention_dairy[c],
            "cmt_mention_product": df.cmt_mention_product[c],
            "cmt_mention_brand": df.cmt_mention_brand[c]
        }
        listofdict.append(info)
    return listofdict

In [30]:
def generalize_keyword(wtkn):  
    ret_wtkn = wtkn
    for i,each in enumerate(ret_wtkn,0):
        if each in ['ดัชมิลล์','ดัชมิล','ดัชมิว','dutchmill','dutch mill','dutch milk','dutchmilk','duchmill','duchie-bio','ดัชชี่','ducthmill','duthmilkselect']:
            ret_wtkn[i] = 'ดัชมิลล์'
        elif each  in ['เมจิ','ซีพีเมจิ','ซีพี เมจิ','cp meiji','meiji','meji','miji','maiji','mejii','cpmeijicom','cpmeijitensai','cpmeijithailand']:
            ret_wtkn[i] = 'ซีพีเมจิ'
        elif each  in ['โฟร์โมสต์','โฟร์โมสท์','โฟร์โมส','โฟโมสต์','โฟโมต','โฟรโมสต์','โฟรโมสต','โฟโมส','โฟโม้ส','foremostfamily','foremost']:
            ret_wtkn[i] = 'โฟร์โมสต์'
        elif each  in ['แดรี่โฮม','dairy home','dairyhome','เดรี่โฮม','เดลี่โฮม']:
            ret_wtkn[i] = 'แดรี่โฮม'
        elif each  in ['โชคชัย','อืมม มิลค์','umm milk','นมฟาร์มโชคชัย','ummilk','ummmilk']:
            ret_wtkn[i] = 'โชคชัย'
        elif each  in ['เอ็มมิลค์','mmilk','เอ็มมิ้ลค์','เอ็มมิลล์','m milk']:
            ret_wtkn[i] = 'เอ็มมิลค์'
        elif each  in ['ไทยเดนมาร์ค','ไทยเดนมาร์ก','ไทย เดนมาร์ค','ไทย เดนมาร์ก','นมวัวแดง']:
            ret_wtkn[i] = 'ไทยเดนมาร์ค'
        elif each  in ['สตอเบอร์รี่','สตรอว์เบอร์รี','สตรอว์เบอร์รี่','สตรอเบอร์รี','รสสตรอเบอรี่','สตรอเบอรี่','สตรอวเบอรี่','สตอเบอรี่','สตรอเบอรี่','สตรอเบอร์รี่','สตรอเบอรี']:
            ret_wtkn[i] = 'สตรอว์เบอร์รี'
        elif each  in ['ชอคโกแล็ต','ช็อกโกแลต','ช็อคโกแลต','ช๊อกโกแลต','ช็อคโกแล็ต','ช็อกโกเลต','ช็อค']:
            ret_wtkn[i] = 'ช็อกโกแลต'
        elif each  in ['มิ้นท์ชอค','มิ้นต์ช้อก','ช็อคโกแลตมินต์','มินต์ช็อคโกแลต']:
            ret_wtkn[i] = 'ช็อกโกแลตมินต์'
        elif each  in ['ไขมันต่ำ','low fat','พร่องมันเนย']:
            ret_wtkn[i] = 'ไขมันต่ำ'
        elif each  in ['ไขมัน 0','ไขมัน 0%','0 fat','0% fat','ขาดมันเนย','nonfat']:
            ret_wtkn[i] = 'ไขมัน 0%'
        elif each  in ['high protein','hi protein','ไฮโปรตีน']:
            ret_wtkn[i] = 'ไฮโปรตีน'
        elif each  in ['free lactose','lactose free','ฟรีแลคโตส','แลคโตสฟรี','lactosefree']:
            ret_wtkn[i] = 'นมฟรีแลคโตส'
        elif each  in ['พาส','พาสเจอไรซ์','พาสเจอร์ไรซ์','พาสเจอร์ไรส์','พาซเจอไรซ์','พาสเจอร์ไลท์']:
            ret_wtkn[i] = 'พาสเจอร์ไรส์'
        elif each  in ['เมจิโกลด์ แม็กซ์','เมจิโกลด์','เมจิ โกลด์','เมจิโกล์ดแม็กซ์','gold','gold max']:
            ret_wtkn[i] = 'เมจิโกลด์'
        elif each  in ['ฝาน้ำเงิน','รสจืด']:
            ret_wtkn[i] = 'รสจืด'
        elif each  in ['ท็อปส์','ทอปส์','ท้อปส์','ท๊อปส์','tops']:
            ret_wtkn[i] = 'Tops'
        elif each  in ['แมคโคร','แม็คโคร','makro']:
            ret_wtkn[i] = 'Makro'
        elif each  in ['โลตัส','lotus']:
            ret_wtkn[i] = 'Lotus'
        elif each  in ['บิ๊กซี','bigc','big c']:
            ret_wtkn[i] = 'BigC'
        elif each in ['ดาร์คช็อกโกแลต','ดาร์กช็อก','ดาร์กช็อกโกแลต']:
            ret_wtkn[i] = 'ดาร์คช็อกโกแลต'
        elif each in ['bedtime','bed time','เบดไทม์']:
            ret_wtkn[i] = 'เบดไทม์'
        elif each in ['เมจิ บัลแกเรีย','เมจิบัลแกเรีย','บัลแกเรีย','bulgaria']:
            ret_wtkn[i] = 'เมจิบัลแกเรีย'
        elif each in ['โยเกิร์ต','โยเกิต','โยเกิรต','โยเกิร์ตพร้อมดื่ม','yoghurt']:
            ret_wtkn[i] = 'โยเกิร์ต'
        elif each in ['7 Eleven','7 eleven','เซเว่น อีเลฟเว่น','เซเว่นอีเลฟเว่น','เซเว่น','เซเวน','เซ่เว่น','7 11']:
            ret_wtkn[i] = '7-Eleven'
        elif each in ['บีทาเก้น','บีทาเกน']:
            ret_wtkn[i] = 'บีทาเก้น'
        elif each in ['โปร','โปรโมชั่น']:
            ret_wtkn[i] = 'โปรโมชั่น'
        elif each in ['นมวัว','นมโค','นมโคสด','น้ำนมโค','นมโคแท้']:
            ret_wtkn[i] = 'นมโค'
        elif each in ['แลกโตส','แล็กโต๊ส','แลคโตส','แลกโทส','แล็คโต๊ส','lactose']:
            ret_wtkn[i] = 'แลคโตส' 
        elif each in ['แพ็ก','แพกเกจ','แพ็กเกจ','แพ็ค','แพ๊ค','แพคเกจ','แพ็คเกจ','บรรจุภัณฑ์']:
            ret_wtkn[i] = 'บรรจุภัณฑ์'
        elif each in ['แคลเซียม','แคลเซี่ยม','calcium']:
            ret_wtkn[i] = 'แคลเซียม'
        elif each in ['nondairy','non dairy','Non-dairy whipping cream']:
            ret_wtkn[i] = 'Non-dairy whipping cream'
        elif each in ['central','เซ็นทรัล']:
            ret_wtkn[i] = 'เซ็นทรัล'
        elif each in ['เลี้ยงเด็ก','เลี้ยงลูก','คนเลี้ยงลูก']:
            ret_wtkn[i] = 'เลี้ยงลูก'
        elif each in ['whey','เวย์','เวย์โปรตีน']:
            ret_wtkn[i] = 'เวย์โปรตีน'
        elif each in ['enzyme','เอนไซม์']:
            ret_wtkn[i] = 'เอนไซม์'
        elif each in ['เอสเปรสโซ','เอสเปรสโซ่','เอสเพรสโซ่','espresso']:
            ret_wtkn[i] = 'เอสเพรสโซ่'
        elif each in ['calories','แคลอรี่']:
            ret_wtkn[i] = 'แคลอรี่'
        elif each in ['maxvalu']:
            ret_wtkn[i] = 'แม็กซ์แวลู'
        elif each in ['แมกนีเซียม','แม็กนีเซียม']:
            ret_wtkn[i] = 'แมกนีเซียม'
        elif each in ['lcarnetine','lcarnitine','แอลคานีทีน']:
            ret_wtkn[i] = 'แอลคานีทีน'
        elif each in ['protein','proteins','protien']:
            ret_wtkn[i] = 'โปรตีน'
        elif each in ['omaga','โอเมกา','โอเมก้า']:
            ret_wtkn[i] = 'โอเมก้า'
        elif each in ['ไอติม','ไอศกรีม','ไอศครีม']:
            ret_wtkn[i] = 'ไอศกรีม'
        elif each in ['amino','อะมิโน','กรดอะมิโน']:
            ret_wtkn[i] = 'กรดอะมิโน'
        elif each in ['grass fed','grassfed','กลาสเฟด']:
            ret_wtkn[i] = 'กลาสเฟด'
        elif each in ['กาแลกโตส','กาแลคโตส','กาแล็กโทส','galactose']:
            ret_wtkn[i] = 'กาแลคโตส'
        elif each in ['โคเลสเตอรอล','ครอเลสฯ','คลอเรสเตอรอล','คลอเลสเตอรอล','คอเลสเตอรอล']:
            ret_wtkn[i] = 'คอเลสเตอรอล'
        elif each in ['cappucino','คาปูชิโน','คาปูชิโน่']:
            ret_wtkn[i] = 'คาปูชิโน'
        elif each in ['ซับพลายเออร์','ซัพพลายเออร์','supplier','suppliers']:
            ret_wtkn[i] = 'ซัพพลายเออร์'
        elif each in ['บาริสตา','บาริสต้า','barista']:
            ret_wtkn[i] = 'บาริสต้า'
        elif each in ['ห้าง','supermarket','ซุปเปอร์','ซุปเปอร์มาร์เกต','ห้างสรรพสินค้า']:
            ret_wtkn[i] = 'ห้างสรรพสินค้า'
        elif each in ['amazon','อเมซอน']:
            ret_wtkn[i] = 'อเมซอน'
        elif each in ['organic','organics','ออแกนิค','ออแกนิก','ออร์แกนิค']:
            ret_wtkn[i] = 'ออร์แกนิค'
        elif each in ['อัลมอนด์','อัลมอล','อัลมอลด์']:
            ret_wtkn[i] = 'อัลมอนด์'
        elif each in ['อ้วก','อ๊วก','อาเจียน']:
            ret_wtkn[i] = 'อาเจียน'
    return ret_wtkn

In [31]:
def find_brands_product(x,catType):
    ret_wtkn = []
    brands = ['ดัชมิลล์','ซีพีเมจิ','โฟร์โมสต์','โชคชัย','แดรี่โฮม','เอ็มมิลค์','แมคโนเลีย','ไทยเดนมาร์ค','หนองโพ','คาเนชั่น','บีทาเก้น','จิตรลดา']
    products = ['สตรอว์เบอร์รี','ช็อกโกแลต','รสกาแฟ','รสหวาน','รสจืด','ไขมันต่ำ','ไขมัน 0%','ไฮโปรตีน','อัลมอนด์'
                ,'รสกล้วย','grass fed','นมฟรีแลคโตส','เมจิโกลด์','นมฮอกไกโด','เบดไทม์','ดาร์คช็อกโกแลต','ไฮแคลเซียม'
                ,'คาราเมล','มอลต์','เมล่อน','ชาเขียวมัจฉะ','บัลแกเรีย','รสธรรมชาติ','รสกลมกล่อม','ซากุระ','วิปครีม']
    milk_kind=['นม','นมข้น','นมจืด','นมสด','นมโค','นมแพะ','นมวัว','นมแม่','หย่านม','กินนม','ดื่มนม','นมขวด','ขวดนม','นมกล่อง','ผลิตภันท์นม','น้ำนมโค'
                     ,'โยเกิร์ต','นมเปรี้ยว','uht','นมถั่วเหลือง','นมผง','พาสเจอร์ไรส์','nondairy','นมโรงเรียน']
    
    choiceList = []
    if catType == 'b':
        choiceList = brands
    elif catType == 'm':
        choiceList = milk_kind
    elif catType == 'p':
        choiceList = products
        
    for i,each in enumerate(x,0):
        if each in choiceList:
            ret_wtkn.append(each)
    return ret_wtkn

#### Copy ชุดคำจาก File มา Process ก่อนลง MongoDB

In [32]:
df_process_comment = df_Corpus.reset_index().copy()
df_process_thread = df_thread.reset_index().copy()

2.1) Tokenize comment

In [33]:
df_process_comment['token_text'] = df_process_comment['text'].apply(lambda x: tokenize_custom(x))

In [34]:
df_process_comment.head()

,commentId,URLs,headline,text,article_date,token_text,Retrived_date
0,1,https://pantip.com/topic/39868603,นมเมจิรสหวานทำไมหายากจังคะ,เมื่อก่อนหลายปีมาแล้ว ในเซเว่นยังมีนมเมจิรสหวา...,2020-05-05 13:22:59,"[ปี, เซเว่น, ยังมี, นม, เมจิ, รสหวาน, ขาย, อยู...",2021-03-22 21:00:24.737
1,2,https://pantip.com/topic/39868603,นมเมจิรสหวานทำไมหายากจังคะ,เบาหวานครับ แฮ่ๆ,2020-05-05 13:22:59,"[เบาหวาน, แฮ่]",2021-03-22 21:00:24.737
2,3,https://pantip.com/topic/39868603,นมเมจิรสหวานทำไมหายากจังคะ,หาซื้อไม่ได้นี่คิดได้ 2 อย่างนะ. คนไม่ค่อย...,2020-05-05 13:22:59,"[หาซื้อไม่ได้, คิดได้, , คน, กินกัน, ทำ, คน, น...",2021-03-22 21:00:24.737
3,4,https://pantip.com/topic/39868603,นมเมจิรสหวานทำไมหายากจังคะ,เป็นรสที่ขายไม่ดีครับ\nรสพื้นฐานคือ\nจืด แบ่งย...,2020-05-05 13:22:59,"[รส, ขายไม่ดี, รส, พื้นฐาน, จืด, แบ่ง, low fat...",2021-03-22 21:00:24.737
4,5,https://pantip.com/topic/39868603,นมเมจิรสหวานทำไมหายากจังคะ,ถ้าขายไม่ดีเค้าก็เอาออกครับ shelf life มันน้อย...,2020-05-05 13:22:59,"[ขายไม่ดี, เค้า, shelf life, เวลา, หมดอายุ, ค่...",2021-03-22 21:00:24.737


2.2) Generalized keyword (Brand & product)

In [35]:
df_process_comment['token_text'] = df_process_comment['token_text'].apply(lambda x: generalize_keyword(x))

In [36]:
df_process_comment['cmt_mention_dairy'] = df_process_comment['token_text'].apply(lambda x: find_brands_product(x,'m'))
df_process_comment['cmt_mention_brand'] = df_process_comment['token_text'].apply(lambda x: find_brands_product(x,'b'))
df_process_comment['cmt_mention_product'] = None
#df_process_comment['cmt_mention_product'] = df_process_comment['token_text'].apply(lambda x: find_brands_product(x,'p'))

2.3) Insert comments to DB

In [37]:
col_comment.delete_many({})
process_data = create_InsertMany_comments(df_process_comment)
col_comment.insert_many(process_data)

2.4) Tokenize head of thread

In [38]:
df_process_thread['token_headline'] = df_process_thread['headline'].apply(lambda x: tokenize_custom(x))

2.5) Generalized keyword (Brand & product)

In [39]:
df_process_thread['token_headline'] = df_process_thread['token_headline'].apply(lambda x: generalize_keyword(x))

2.6) Mark domain/entity mention

In [40]:
df_process_thread['t_mention_dairy'] = df_process_thread['token_headline'].apply(lambda x: find_brands_product(x,'m'))
df_process_thread['t_mention_brand'] = df_process_thread['token_headline'].apply(lambda x: find_brands_product(x,'b'))
df_process_thread['t_mention_product'] = None
#df_process_thread['t_mention_product'] = df_process_thread['token_headline'].apply(lambda x: find_brands_product(x,'p'))

In [41]:
df_process_thread.head()

,commentId,URLs,headline,token_headline,mention_product,mention_brand,t_mention_dairy,t_mention_brand,t_mention_product
0,1,https://pantip.com/topic/39868603,นมเมจิรสหวานทำไมหายากจังคะ,"[นม, ซีพีเมจิ, รสหวาน, หายาก]",None,None,[นม],[ซีพีเมจิ],None
1,8,https://pantip.com/topic/39747201,ทำไมนมเมจิขวดใหญ่ 2 ลิตร ไม่มีซีลที่ปากแบบขวดเล็ก,"[นม, ซีพีเมจิ, ขวด, ลิตร, ซีล, ปาก, ขวด]",None,None,[นม],[ซีพีเมจิ],None
2,9,https://pantip.com/topic/39597709,ระหว่างนมเมจิต้มกับนมเมจิที่ไม่ได้ต้ม เมื่อนำไ...,"[นม, ซีพีเมจิ, ต้ม, นม, ซีพีเมจิ, ไม่ได้, ต้ม,...",None,None,"[นม, นม]","[ซีพีเมจิ, ซีพีเมจิ]",None
3,15,https://pantip.com/topic/39928742,ติดนมมาก ทำยังไงดี?,"[ติด, นม, ทำ, ดี]",None,None,[นม],[],None
4,26,https://pantip.com/topic/40190870,สูง 147 ม.1 มีสิทธิสูงเพิ่มไหมครับ,"[สูง, สิทธิ, สูง, ไหม]",None,None,[],[],None


2.7) Insert thread to DB

In [42]:
col_thread.delete_many({})
process_data = create_InsertMany_Thread(df_process_thread)
col_thread.insert_many(process_data)

In [43]:
del df_thread,df_Corpus,df_process_comment,df_process_thread

### 4.สรุป Term & reduced_keyword

In [8]:
execption_urls = ['https://pantip.com/topic/30833944','https://pantip.com/topic/30105850','https://pantip.com/topic/35439062']

In [9]:
attrributes = ['เอ็มมิลค์','แดรี่โฮม','แมคโนเลีย','โชคชัย','โฟร์โมสต์','ไทยเดนมาร์ค','ไวตามิลค์','จิตรลดา','ซีพีเมจิ','ดัชมิลล์','หนองโพ'
               ,'7-Eleven','bigc','iso','lotus','makro','non-dairy whipping cream','tops','uht','vitamin','whipping','เก็บรักษา','เข้มข้น','เครื่องดื่มให้พลังงาน','เจมส์จิ'
               ,'เจริญอาหาร','เจลลาติน','เชื้อโรค','เชื้อรา','เซ็นทรัล','เต้าหู้','เต้าฮวย','เต้าฮวยนมสด','เท','เทนนิส','เทน้ำเทท่า','เนย','เนยสด','เนื้อไก่','เนื้อสัตว์','เบเกอรี่'
               ,'เบดไทม์','เป็นหวัด','เปรี้ยว','เพิ่มปริมาณ','เพื่อนรุ่นเดียวกัน','เฟซบุ้ค','เมจิโกลด์','เมจิไขมัน 0%','เมจิบัลแกเรีย','เมล่อน','เว็บไซต์','เหม็น','เอากำไร'
               ,'แก้ขัด','แกลลอน','แคนตาลูป','แครอท','แช่','แช่เย็น','แน่นท้อง','แนะนำ','แบบสอบถาม','แป้งข้าวโพด','แป้งสาลี','แพะ','แมกโนเลีย','แม็กซ์แวลู'
               ,'แลคตาซอย','แล็ป','แอลมอนด์','โกโก้','โกลเดนเพลส','โคนม','โครงการส่วนพระองค์','โครงการหลวง','โซดา','โฟมนม','โรงเรียน','โรงงาน','โรงพยาบาล'
               ,'โรบินสัน','โอวัลติน','โฮมเมด','ไข่','ไข่ไก่','ไขกระดูก','ไข่ขาว','ไข่ดาว','ไข่ต้ม','ไต','ไมโครเวฟ','ไม่ชอบ','ไม่อร่อย','ไหว้เจ้า','ไอศกรีม','กระทรวงพาณิชย์'
               ,'กระทรวงสาธารณสุข','กลมกล่อม','กล้วย','กล้วยน้ำว้า','กล้วยหอม','กล่อง','กลางดึก','กลาสเฟด','กลิ่น','กลิ่นคาว','การจัดส่ง','การบรรจุ','การปรุง','กินเจ'
               ,'กินรวบ','ขโมย','ขวด','ขวดนม','ของหวาน','ขาดตลาด','ข้าวเหนียว','ข้าวโพด','ข้าวโอ๊ต','ข้าวตัง','ข้าวสาลี','คนแก่','คนไข้','คนวัยหนุ่มสาว','คนสูงอายุ'
               ,'ครีมเทียม','ความแตกต่าง','คอนเฟลก','ค่าขนส่ง','คาราเมล','คาว','คุณหมอ','งานวิจัย','จิฟฟี่','จืดชืด','จุดขาย','ช็อกโกแลต','ช็อกโกแลตมิ้นต์','ชั่งใจ','ชา'
               ,'ชาเขียว','ชาเขียวมัจฉะ','ชาเมะมัทฉะ','ชาเย็น','ชาไทย','ชีวิตประจำวัน','ซัพพลายเออร์','ซากุระ','ซีเรียล','ซีพี','ดัชมิลล์','ดาร์คช็อกโกแลต','ดื่มนม','ต้ม'
               ,'ตัวแทนจำหน่าย','ติดใจ','ตีฟอง','ตู้เย็น','ถั่วเหลือง','ถุงพลาสติก','ถูกปาก','ถูกอกถูกใจ','ทวิตเตอร์','ท้องตลาด','ทำเงิน','ทำกำไร','ทำกิจกรรม'
               ,'ทำความสะอาด','ธัญพืช','นม','นมแพะ','นมโค','นมโรงเรียน','นมกล่อง','นมขวด','นมชง','นมถั่วเหลือง','นมผง','นมฟรีแลคโตส','นมยูเอชที','นมสด'
               ,'นมอัดเม็ด','นมอุ่น','น้ำเต้าหู้','น้ำนม','น้ำผลไม้','น้ำย่อย','น้ำหนัก','น้ำอัดลม','บรรจุขวด','บลูเบอร์รี่','บัตเตอร์','บาริสต้า','ประโยชน์','ประกอบอาหาร','ป่วย'
               ,'ผลไม้','ผลวิจัย','ผลิต','ผสม','ผู้สูงอายุ','ฝืดคอ','พนักงานขาย','พนักงานขายนม','พยาบาล','พระราชดำริ','พรีเซนเตอร์','พาสเจอร์ไรส์','ฟรอยด์','ฟอง'
               ,'ฟองเต้าหู้','ฟองนม','ฟองมัน','มหาลัย','มอลต์','มะพร้าวอ่อน','มะม่วง','มะละกอ','มังสวิรัติ','มัทฉะ','มันเทศ','มิลค์เชค','มีชื่อเสียง','มีประโยชน์','มื้อเช้า','มูส'
               ,'ยี่ห้อ','ยูเอชที','รสกล้วย','รสกาแฟ','รสจืด','รสชาติ','รสธรรมชาติ','รสหวาน','ร้านเบเกอรี่','ร้านกาแฟ','ร้านขายของชำ','ร้านค้า','ร้านนม','ร้านนมสด','รีดนม'
               ,'ลอง','ลาซาด้า','ลำไส้','ลูกเดือย','ลูกสุนัข','ลูกหมา','วอลนัท','วัยเด็ก','วัยเด็กเล็ก','วัวนม','วานิลลา','วาฟเฟิล','สเตรปโตคอคคัส','สเตอริไลซ์','สตรอว์เบอร์รี่'
               ,'สตาร์บัค','สบู่','สมูทตี้','สยามพารากอน','สลัด','ส่วนแบ่งการตลาด','ส่วนผสม','สะดวก','สารกันบูด','สูตรลดน้ำตาล','หมา','หลอด','หวาน','ห้องเย็น','หอม'
               ,'ห้างสรรพสินค้า','อเมซอน','อโวคาโด','อกไก่','อร่อย','ออร์แกนิค','อัลมอนด์','อาหารเจ','อาหารเสริม','อาหารว่าง','อีโคไล','อุตสาหกรรม','อุ่นนม','ฮอร์โมน'
               ,'ฮั่งก้วย','ฮาลาล','หางนม','เด็ก','เด็กเล็ก','เด็กแรกเกิด','เด็กโต','เด็กผู้ชาย','เด็กวัยเรียน','เด็กอ่อน','เลี้ยงลูก','แปรงฟัน','แพ้ท้อง','แม่','แม่ลูกอ่อน','ให้นม'
               ,'คนท้อง','คลอด','คลอดลูก','คุณแม่','ชงนม','ตั้งครรภ์','ตั้งท้อง','ท้อง','นมแม่','ประถม','ฝากครรภ์','ฝากท้อง','มีครรภ์','มีครอบครัว','มีลูก','ลูก','ลูกชาย'
               ,'ลูกสาว','หย่านม','แถม','แพคคู่','โปรโมชั่น','โปรโมต','การลดราคา','ของแถม','ชิม','ลดราคา','อาหาร','เอสเพรสโซ่','โอวัลติน','โฮลวีท','ไมโล','กาแฟ'
               ,'กาแฟเย็น','กาแฟร้อน','กาแฟสด','ขนม','ขนมเค้ก','ขนมปัง','ขนมหวาน','คาปูชิโน','ชง','ชงกาแฟ','ทำฟอง','ทำอาหาร','มอคค่า','ลาเต้','อเมริกาโน'
               ,'haccp','เหม็นเปรี้ยว','เหม็นบูด','แบคทีเรีย','กระบวนการผลิต','การบูด','ของเสีย','ความสะอาด','คุณภาพ','ตะกอน','บรรจุภัณฑ์','บุบ','บูด','ปนเปื้อน','ปริมาตร'
               ,'วันหมดอายุ','สะอาด','สารเคมี','สารสังเคราะห์','สิ่งแปลกปลอม','สิ่งปลอมปน','หมดอายุ','ความคุ้มค่า','คุ้มค่า','ราคาแพง','ราคาถูก','dha','เวย์โปรตีน'
               ,'แคลเซียม','แป้ง','แมกนีเซียม','แลคโตส','แลคติก','แอนโดรเจน','แอนตี้ไบโอติก','แอลคานีทีน','โซเดียม','โปรตีน','โพแทสเซียม','โฟเลต','โอเมก้า','ไขมัน'
               ,'ไมอิลีน','ไอโอดีน','กรดอะมิโน','กลูโคส','กาเฟอีน','กาแลกโตส','กาแลคโตส','คาเฟอีน','คาร์โบไฮเดรต','ซูโครส','ธาตุเหล็ก','ธาตุอาหาร','น้ำ','น้ำตาล','น้ำตาลเทียม'
               ,'น้ำตาลทราย','น้ำตาลทรายแดง','วิตามิน','สารอาหาร','อาร์จีนีน','whey formula','เครียด','เตี้ย','เบาหวาน','เพาะกาย','เพิ่มน้ำหนัก','เล่นเวท','เล่นกล้าม'
               ,'เวย์โปรตีน','เสริมสร้างกระดูก','เสริมสร้างร่างกาย','เอนไซม์','แข็งแรง','แคลอรี่','แพ้','แพ้นม','โด๊ป','โภชนาการ','โรคเบาหวาน','โรคโลหิตจาง','โรคประจำตัว'
               ,'โรคมะเร็ง','โลหิตจาง','ไขมัน 0%','ไขมันต่ำ','ไขมันทรานส์','ไฮแคลเซียม','ไฮโปรตีน','กรดไขมัน','กรดอะมิโน','กระเพาะ','กระเพาะอาหาร','กระดูก','กระดูกแข็งแรง'
               ,'กระดูกพรุน','กระดูกสันหลัง','กระดูกอ่อน','กล้าม','กล้ามเนื้อ','การเจริญเติบโต','การเล่นกีฬา','การฉายแสง','การย่อยอาหาร','การลดน้ำหนัก','การออกกำลังกาย'
               ,'ขับถ่าย','ขาดสารอาหาร','คลื่นไส้','ความดัน','ความสูง','ความหวาน','ความอ้วน','คอเลสเตอรอล','คอลลาเจน','คุณประโยชน์','คุมอาหาร','ช่องท้อง','ชั่งน้ำหนัก'
               ,'ซ่อมแซมส่วนที่สึกหรอ','ดูแลสุขภาพ','ตัวเตี้ย','ตัวเล็ก','ตัวสูง','ท้องเสีย','ท้องผูก','ท้องร่วง','ท้องว่าง','ท้องอืด','นอนหลับ','นักโภชนาการ','นักกีฬา','บำรุง','บำรุงกำลัง'
               ,'ปวดท้อง','ผิดปกติ','พลังงาน','พิษสุนัขบ้า','ฟันผุ','ฟิต','ฟิตเนส','ฟื้นฟูสภาพ','ภูมิแพ้','มวลกระดูก','มะเร็ง','ระบบขับถ่าย','รักษาสุขภาพ','ลดความอ้วน','ลดน้ำหนัก'
               ,'ส่วนสูง','สุขภาพ','สุขอนามัย','สูง','หลังอาหาร','หายป่วย','อดอาหาร','อยากสูง','ออกกำลัง','ออกกำลังกาย','อะมิโน','อักเสบ','อัลบูมิน','อาเจียน','เป็นประจำ','ไม่คุ้มค่า'
               ,'ขาย','ขายไม่ดี','ขายไม่ออก','ขายดี','ขายปลีก','ขายส่ง','ซื้อประจำ','หาซื้อ','หายาก','หาซื้อไม่ได้']

stores = ['tops','makro','lotus','bigc','7-Eleven']
#products = ['สตรอว์เบอร์รี','ช็อกโกแลต','รสกาแฟ','รสหวาน','รสจืด','ไขมันต่ำ','ไขมัน 0%','ไฮโปรตีน','อัลมอนด์'
#                ,'รสกล้วย','grass fed','นมฟรีแลคโตส','เมจิโกลด์','นมฮอกไกโด','เบดไทม์','ดาร์คช็อกโกแลต','ไฮแคลเซียม'
#                ,'คาราเมล','มอลต์','เมล่อน','ชาเขียวมัจฉะ','บัลแกเรีย','รสธรรมชาติ','รสกลมกล่อม','ซากุระ']
#milk_kind=['นม','นมข้น','นมจืด','นมสด','กินนม','ดื่มนม','ขวดนม','นมวัว','นมกล่อง','ผลิตภันท์นม','น้ำนมโค'
#                     ,'โยเกิร์ต','นมเปรี้ยว','uht','นมถั่วเหลือง','นมผง','พาสเจอร์ไรส์']

#  EDA ใน Excel ได้ flavor 68 ตัว (ถ้าดึงยี่ห้ออื่นก็เพิ่มอีก)
#avai_flavs = ['เมจิเมจิโกลด์','เมจิเมล่อน','เมจิไขมัน 0%','เมจิไขมันต่ำ','เมจิไฮโปรตีน','เมจิช็อกโกแลต','เมจิชาเขียวมัจฉะ','เมจิดาร์คช็อกโกแลต'
# ,'เมจินมฟรีแลคโตส','เมจิบัลแกเรีย','เมจิมอลต์','เมจิรสกลมกล่อม','เมจิรสกล้วย','เมจิรกาแฟ','เมจิรสจืด','เมจิรสธรรมชาติ','เมจิรสหวาน'
# ,'เมจิสตรอว์เบอร์รี','เมจิอัลมอนด์','เอ็มมิลค์นมฟรีแลคโตส','เอ็มมิลค์รสจืด','แดรี่โฮมgrass fed','แดรี่โฮมเบดไทม์','แดรี่โฮมช็อกโกแลต'
# ,'แดรี่โฮมรสกล้วย','แดโฮมรสจืด','แดรี่โฮมรสหวาน','แดรี่โฮมสตรอว์เบอร์รี','แมคโนเลียไขมันต่ำ','แมคโนเลียช็อกโกแลต','แมคโนเลียรสจืด'
# ,'โชคชัยไขมันต่ำ','โชคชัยช็อกโกแลต','โชคชัยรสกาแฟ','โชคชัยรสจืด','โชคชัยสตรอว์เบอร์รี','ฟร์โมสต์ไขมัน 0%','โฟร์โมสต์ไขมันต่ำ'
# ,'โฟร์โมสต์คาราเมล','โฟร์โมสต์ช็อกโกแลต','โฟร์โมสต์รสกาแฟ','โฟร์โมสต์รสจืด','โฟร์โมสต์สตรอว์เบอร์รี','ไทยเดนมาร์คช็อกโกแลต'
# ,'ไทยเดนมาร์ครสกาแฟ','ไทยเดนมาร์ครสจืด','ไทยเดนมาร์ครสหวาน','ไทยเดนมาร์คสตรอว์เบอร์รี','คาเนชั่นรสจืด','จิตรลดาช็อกโกแลต'
# ,'จิตรลดารสจืด','จิตรลดารสหวาน','จิตรลดาสตรอว์เบอร์รี','ดัชมิลล์ไขมัน 0%','ดัชมิลล์ไขมันต่ำ','ดัชมิลล์ไฮโปรตีน','ดัชลล์อกกแลต'
# ,'ดัชมิลล์มอลต์','ดัชมิลล์รสกาแฟ','ดัชมิลล์รสจืด','ดัชมิลล์สตรอว์เบอร์รี','หนองโพไขมัน 0%','หนองโพไขมันต่ำ','หนองโพช็อกโกแลต'
# ,'หนองโพรสกาแฟ','หนองโพรสจืด','หนองโพรสหวาน','หนองโพสตรอว์เบอร์รี']

reduceCol = attrributes# + stores 
#reduceCol_all = attrributes + stores + milk_kind + products

In [10]:
def reduced_keyword(wtkn,redCol):
    del_list = []
    ret_wtkn = wtkn
    for each in ret_wtkn:
        if each not in redCol:
            del_list.append(each)
    ret_wtkn = [x for x in ret_wtkn if x not in del_list]
    return ret_wtkn

In [11]:
def freq_brand(x):
    #e = eval(x)
    e = x
    e.sort()
    f = [(k,len(list(g))) for k, g in groupby(e)]
    return f

In [12]:
def concat_flavor(x,y):
    listflav = []
    for i in x:
        for j in y:
            listflav.append(i+j)
    return listflav

### 5.สร้าง (Reduce) Bag of word ด้วย dictionary.doc2bow จัดลง dataframe<br>
คัดเลือกกระทู้และความเห็นที่เกี่ยวกับนมในไฟล์ 03.EDA_Headline_Comment ก่อน จัดเก็บลง MongoDB แล้วจึงทำส่วนนี้

In [13]:
cursor_clean = col_claen.find()
df_join_url = pd.DataFrame(cursor_clean)

In [14]:
df_join_url

,_id,commentId,URLs,token_headline,docs_mention_domain,docs_mention_brand,headline,article_date,text,token_text,token_text_cnt
0,606f2cb7592106280e46e872,0,https://pantip.com/topic/39868603,"[นม, ซีพีเมจิ, รสหวาน, หายาก]","[[นม, 2]]","[[ซีพีเมจิ, 1]]",นมเมจิรสหวานทำไมหายากจังคะ,2020-05-05 13:22:59,เมื่อก่อนหลายปีมาแล้ว ในเซเว่นยังมีนมเมจิรสหวา...,"[ปี, 7-Eleven, ยังมี, นม, ซีพีเมจิ, รสหวาน, ขา...","[[7-Eleven, 1], [Tops, 1], [กลาง, 1], [ขวด, 2]..."
1,606f2cb7592106280e46e873,1,https://pantip.com/topic/39868603,"[นม, ซีพีเมจิ, รสหวาน, หายาก]","[[นม, 1]]","[[ซีพีเมจิ, 1]]",นมเมจิรสหวานทำไมหายากจังคะ,2020-05-05 13:22:59,เบาหวานครับ แฮ่ๆ,"[เบาหวาน, แฮ่]","[[เบาหวาน, 1], [แฮ่, 1]]"
2,606f2cb7592106280e46e874,2,https://pantip.com/topic/39868603,"[นม, ซีพีเมจิ, รสหวาน, หายาก]","[[นม, 1]]","[[ซีพีเมจิ, 1]]",นมเมจิรสหวานทำไมหายากจังคะ,2020-05-05 13:22:59,หาซื้อไม่ได้นี่คิดได้ 2 อย่างนะ. คนไม่ค่อย...,"[หาซื้อไม่ได้, คิดได้, , คน, กินกัน, ทำ, คน, น...","[[ขาย, 1], [คน, 2], [นิยม, 1], [รสหวาน, 1], [,..."
3,606f2cb7592106280e46e875,3,https://pantip.com/topic/39868603,"[นม, ซีพีเมจิ, รสหวาน, หายาก]","[[นม, 1]]","[[ซีพีเมจิ, 1]]",นมเมจิรสหวานทำไมหายากจังคะ,2020-05-05 13:22:59,เป็นรสที่ขายไม่ดีครับ\nรสพื้นฐานคือ\nจืด แบ่งย...,"[รส, ขายไม่ดี, รส, พื้นฐาน, จืด, แบ่ง, ไขมันต่...","[[ขายไม่ดี, 1], [คน, 1], [พื้นฐาน, 1], [รส, 2]..."
4,606f2cb7592106280e46e876,4,https://pantip.com/topic/39868603,"[นม, ซีพีเมจิ, รสหวาน, หายาก]","[[นม, 1]]","[[ซีพีเมจิ, 1]]",นมเมจิรสหวานทำไมหายากจังคะ,2020-05-05 13:22:59,ถ้าขายไม่ดีเค้าก็เอาออกครับ shelf life มันน้อย...,"[ขายไม่ดี, เค้า, shelf life, เวลา, หมดอายุ, ค่...","[[ขวด, 1], [ขายไม่ดี, 2], [กี่, 1], [ซื้อ, 1],..."
...,...,...,...,...,...,...,...,...,...,...,...
12541,606f2cb7592106280e47196f,12541,https://pantip.com/topic/30023475,"[นม, ถุง, ขาย, โรงเรียน, ขาย, ที่อื่น, กทม, ไห...","[[นม, 1]]","[[ไทยเดนมาร์ค, 2]]",นมถุง ที่ขายในโรงเรียน มีขายที่อื่นในกทม.อีกไห...,2013-01-07 17:36:47,นมวัวแดง น่าจะเป็นชื่อเรียกของยี่ห้อ ไทย-เดนมา...,"[ไทยเดนมาร์ค, น่าจะเป็น, ชื่อ, ยี่ห้อ, ไทยเดนม...","[[ยี่ห้อ, 1], [ไทยเดนมาร์ค, 2], [วัว, 1], [ชื่..."
12542,606f2cb7592106280e471970,12542,https://pantip.com/topic/30023475,"[นม, ถุง, ขาย, โรงเรียน, ขาย, ที่อื่น, กทม, ไห...","[[นม, 1]]",None,นมถุง ที่ขายในโรงเรียน มีขายที่อื่นในกทม.อีกไห...,2013-01-07 17:36:47,เมื่อก่อนตอนเด็กๆค่จำได้นะว่านมถุงละ5บาทเองอะ\...,"[ตอน, เด็ก, ๆค่, จำได้, นม, ถุง, อะ, ซื้อ, สหก...","[[นม, 1], [ซื้อ, 1], [ตอน, 1], [เด็ก, 1], [อะ,..."
12543,606f2cb7592106280e471971,12543,https://pantip.com/topic/30023475,"[นม, ถุง, ขาย, โรงเรียน, ขาย, ที่อื่น, กทม, ไห...","[[นม, 1]]",None,นมถุง ที่ขายในโรงเรียน มีขายที่อื่นในกทม.อีกไห...,2013-01-07 17:36:47,ตกลงได้ไปซื้อมาแล้วนะคะที่เกษตร หลงทางอยู่ในม....,"[ตกลง, ซื้อ, เกษตร, หลงทาง, เกษตร, แห, เจอ, อร...","[[หาย, 1], [กิน, 1], [ซื้อ, 1], [ขอบคุณ, 1], [..."
12544,606f2cb7592106280e471972,12544,https://pantip.com/topic/30023475,"[นม, ถุง, ขาย, โรงเรียน, ขาย, ที่อื่น, กทม, ไห...","[[นม, 1]]",None,นมถุง ที่ขายในโรงเรียน มีขายที่อื่นในกทม.อีกไห...,2013-01-07 17:36:47,มีค่ะ สนใจติดต่อได้นะคะ ราคากันเองค่ะ \n\t\t\t...,"[สนใจ, ติดต่อ, ข้อความ, เวลา]","[[เวลา, 1], [ข้อความ, 1], [ติดต่อ, 1], [สนใจ, 1]]"


In [15]:
df_join_url['token_text_reduce'] = df_join_url['token_text'].apply(lambda x: reduced_keyword(x, reduceCol))

In [16]:
df_join_url.head(2)

,_id,commentId,URLs,token_headline,docs_mention_domain,docs_mention_brand,headline,article_date,text,token_text,token_text_cnt,token_text_reduce
0,606f2cb7592106280e46e872,0,https://pantip.com/topic/39868603,"[นม, ซีพีเมจิ, รสหวาน, หายาก]","[[นม, 2]]","[[ซีพีเมจิ, 1]]",นมเมจิรสหวานทำไมหายากจังคะ,2020-05-05 13:22:59,เมื่อก่อนหลายปีมาแล้ว ในเซเว่นยังมีนมเมจิรสหวา...,"[ปี, 7-Eleven, ยังมี, นม, ซีพีเมจิ, รสหวาน, ขา...","[[7-Eleven, 1], [Tops, 1], [กลาง, 1], [ขวด, 2]...","[7-Eleven, นม, ซีพีเมจิ, รสหวาน, ขาย, ขวด, ขวด..."
1,606f2cb7592106280e46e873,1,https://pantip.com/topic/39868603,"[นม, ซีพีเมจิ, รสหวาน, หายาก]","[[นม, 1]]","[[ซีพีเมจิ, 1]]",นมเมจิรสหวานทำไมหายากจังคะ,2020-05-05 13:22:59,เบาหวานครับ แฮ่ๆ,"[เบาหวาน, แฮ่]","[[เบาหวาน, 1], [แฮ่, 1]]",[เบาหวาน]


In [17]:
dictionary = gensim.corpora.Dictionary(df_join_url['token_text_reduce'])

In [18]:
df_join_url['reduce_bow'] = df_join_url["token_text_reduce"].map(dictionary.doc2bow)
df_join_url['token_text_reduce_cnt'] = df_join_url["reduce_bow"].apply(lambda x:[(dictionary[id_], frequence) for id_, frequence in x])

In [19]:
df_join_url.drop(columns=['article_date','reduce_bow'],inplace=True)

In [20]:
df_join_url.head(2)

,_id,commentId,URLs,token_headline,docs_mention_domain,docs_mention_brand,headline,text,token_text,token_text_cnt,token_text_reduce,token_text_reduce_cnt
0,606f2cb7592106280e46e872,0,https://pantip.com/topic/39868603,"[นม, ซีพีเมจิ, รสหวาน, หายาก]","[[นม, 2]]","[[ซีพีเมจิ, 1]]",นมเมจิรสหวานทำไมหายากจังคะ,เมื่อก่อนหลายปีมาแล้ว ในเซเว่นยังมีนมเมจิรสหวา...,"[ปี, 7-Eleven, ยังมี, นม, ซีพีเมจิ, รสหวาน, ขา...","[[7-Eleven, 1], [Tops, 1], [กลาง, 1], [ขวด, 2]...","[7-Eleven, นม, ซีพีเมจิ, รสหวาน, ขาย, ขวด, ขวด...","[(7-Eleven, 1), (ขวด, 2), (ขาย, 1), (ขายไม่ดี,..."
1,606f2cb7592106280e46e873,1,https://pantip.com/topic/39868603,"[นม, ซีพีเมจิ, รสหวาน, หายาก]","[[นม, 1]]","[[ซีพีเมจิ, 1]]",นมเมจิรสหวานทำไมหายากจังคะ,เบาหวานครับ แฮ่ๆ,"[เบาหวาน, แฮ่]","[[เบาหวาน, 1], [แฮ่, 1]]",[เบาหวาน],"[(เบาหวาน, 1)]"


Create cooccurence matrix

In [21]:
# comment ออกเมื่อไม่ concat brand+flavor
df_final = df_join_url.copy()
df_final.set_index('commentId',inplace=True)
#df_final['mention_product'] = df_final['mention_product'].apply(lambda x: x if x is None else freq_brand(x))
#df_final['mention_brand'] = df_final['mention_brand'].apply(lambda x: x if x is None else freq_brand(x))
#df_final['mention_domain'] = df_final['mention_domain'].apply(lambda x: x if x is None else freq_brand(x))

In [22]:
df_final.head(2)

,_id,URLs,token_headline,docs_mention_domain,docs_mention_brand,headline,text,token_text,token_text_cnt,token_text_reduce,token_text_reduce_cnt
commentId,,,,,,,,,,,
0,606f2cb7592106280e46e872,https://pantip.com/topic/39868603,"[นม, ซีพีเมจิ, รสหวาน, หายาก]","[[นม, 2]]","[[ซีพีเมจิ, 1]]",นมเมจิรสหวานทำไมหายากจังคะ,เมื่อก่อนหลายปีมาแล้ว ในเซเว่นยังมีนมเมจิรสหวา...,"[ปี, 7-Eleven, ยังมี, นม, ซีพีเมจิ, รสหวาน, ขา...","[[7-Eleven, 1], [Tops, 1], [กลาง, 1], [ขวด, 2]...","[7-Eleven, นม, ซีพีเมจิ, รสหวาน, ขาย, ขวด, ขวด...","[(7-Eleven, 1), (ขวด, 2), (ขาย, 1), (ขายไม่ดี,..."
1,606f2cb7592106280e46e873,https://pantip.com/topic/39868603,"[นม, ซีพีเมจิ, รสหวาน, หายาก]","[[นม, 1]]","[[ซีพีเมจิ, 1]]",นมเมจิรสหวานทำไมหายากจังคะ,เบาหวานครับ แฮ่ๆ,"[เบาหวาน, แฮ่]","[[เบาหวาน, 1], [แฮ่, 1]]",[เบาหวาน],"[(เบาหวาน, 1)]"


In [23]:
#df_brand_cooc = pd.concat([pd.DataFrame(s,columns=['w','c']).set_index('w') for s in df_final["mention_brand"]], axis=1, sort=False).fillna(0).T.set_index(df_final.index)
#df_prd_cooc = pd.concat([pd.DataFrame(s,columns=['w','c']).set_index('w') for s in df_final["mention_product"]], axis=1, sort=False).fillna(0).T.set_index(df_final.index)
df_cooc = pd.concat([pd.DataFrame(s,columns=['w','c']).set_index('w') for s in df_final["token_text_reduce_cnt"]], axis=1, sort=False).fillna(0).T.set_index(df_final.index)

In [24]:
#df_brand_cooc.reset_index(inplace=True)
#df_prd_cooc.reset_index(inplace=True)
df_cooc.reset_index(inplace=True)

In [25]:
df_cooc.head()

,commentId,7-Eleven,ขวด,ขาย,ขายไม่ดี,ซีพีเมจิ,นม,รสหวาน,เบาหวาน,หาซื้อไม่ได้,...,ช่องท้อง,แอนตี้ไบโอติก,ส่วนแบ่งการตลาด,แล็ป,คุมอาหาร,คนไข้,ฟื้นฟูสภาพ,อโวคาโด,เครื่องดื่มให้พลังงาน,โรคโลหิตจาง
0,0,1.0,2.0,1.0,1.0,1.0,2.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
# ทดลอง  QAP mathod case2,4: Brand vs Product vs Attribute (ไม่เอาแล้ว)
#join_df = pd.merge(df_brand_cooc, df_prd_cooc, on='commentId', how='inner', suffixes=(False, False))
#join_df = pd.merge(join_df, df_cooc, on='commentId', how='inner')

# ทดลอง  QAP mathod case3: Product vs Attribute (ไม่เอาแล้ว)
#join_df = pd.merge(df_prd_cooc, df_cooc, on='commentId', how='inner', suffixes=(False, False))

# ทดลอง  QAP mathod case1,5: Brand vs Attribute กรณีแยก ยี่ห้อ และ ผลิตภัณฑ์ จาก reduce_bow_txt
#join_df = pd.merge(df_brand_cooc, df_cooc, on='commentId', how='inner', suffixes=(False, False))

# ไม่ Join ทำในกรณีไม่แยก ยี่ห้อ และ ผลิตภัณฑ์ ออกจาก reduce_bow_txt
join_df = df_cooc.copy()

join_df.set_index('commentId',inplace=True)

In [27]:
join_df.head()

,7-Eleven,ขวด,ขาย,ขายไม่ดี,ซีพีเมจิ,นม,รสหวาน,เบาหวาน,หาซื้อไม่ได้,กาแฟ,...,ช่องท้อง,แอนตี้ไบโอติก,ส่วนแบ่งการตลาด,แล็ป,คุมอาหาร,คนไข้,ฟื้นฟูสภาพ,อโวคาโด,เครื่องดื่มให้พลังงาน,โรคโลหิตจาง
commentId,,,,,,,,,,,,,,,,,,,,,
0,1.0,2.0,1.0,1.0,1.0,2.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Export ข้อมูล เพื่อนำไปสุ่มแถวทำ QAP

In [28]:
join_df.to_csv(root_output_path+"/cooc_bow_all.csv",index=True)

## Source for graph generate

### 1.Create frequency co-occurrence matrix จาก Bag of word<br>
* ใช้สำหรับทำ MDS และโยนเข้า R (ทำ normalized บน R) ในลักษณะ Term-Term Cross tabulation<br> 
(คนละอย่างกับ TF-IDF cooc matrix ที่เป็น Word-word ซึ่งต้องรัน bigram เป็นราย document)

In [28]:
item_item_matrix = pd.DataFrame(index=join_df.columns,columns=join_df.columns).fillna(0)

In [29]:
def cnt_Grpby(df,i,j):
    a = pd.DataFrame(df.iloc[:,i])
    b = pd.DataFrame(df.iloc[:,j])
    slice_df = pd.concat([a,b], axis=1, join="inner")
    slice_df.columns = ['A','B']
    cooc = slice_df[(slice_df.A>0)&(slice_df.B>0)]
    return cooc.shape[0]

In [30]:
# Co-occurrence
for i in range(0,len(item_item_matrix.columns)):
    for j in range(0,len(item_item_matrix.columns)):
        #if i != j:
        item_item_matrix.iloc[i,j] = cnt_Grpby(join_df,i,j)
        #else:
         #   item_item_matrix.iloc[i,j] = 0

In [31]:
item_item_matrix.head()

,7-Eleven,ขวด,ขาย,ขายไม่ดี,ซีพีเมจิ,นม,รสหวาน,เบาหวาน,หาซื้อไม่ได้,กาแฟ,...,ช่องท้อง,แอนตี้ไบโอติก,ส่วนแบ่งการตลาด,แล็ป,คุมอาหาร,คนไข้,ฟื้นฟูสภาพ,อโวคาโด,เครื่องดื่มให้พลังงาน,โรคโลหิตจาง
7-Eleven,181,20,52,3,25,77,8,1,0,4,...,0,0,0,0,0,0,0,0,0,0
ขวด,20,674,110,2,130,385,10,3,1,32,...,0,0,1,0,0,0,0,0,0,0
ขาย,52,110,804,4,81,431,16,6,3,51,...,0,1,2,0,0,0,0,0,0,0
ขายไม่ดี,3,2,4,10,3,2,4,0,0,1,...,0,0,0,0,0,0,0,0,0,0
ซีพีเมจิ,25,130,81,3,765,414,20,2,0,61,...,0,0,0,0,0,0,0,0,0,0


In [32]:
item_item_matrix.to_excel(comment_cooc_freq_file, index=True)

### 2.Create co-occurrence matrix with Jaccard Distance สำหรับสร้างกราฟ

In [56]:
item_item_matrix = pd.DataFrame(index=join_df.columns,columns=join_df.columns).fillna(0)

In [57]:
def jaccard_index(df,i,j):
    a = pd.DataFrame(df.iloc[:,i])
    b = pd.DataFrame(df.iloc[:,j])
    slice_df = pd.concat([a,b], axis=1, join="inner")
    slice_df.columns = ['A','B']
    a_in_b = slice_df[(slice_df.A>0)&(slice_df.B>0)]
    a_occur = slice_df[(slice_df.A>0)]
    b_occur = slice_df[(slice_df.B>0)]
    jacc_index = a_in_b.shape[0]/(a_occur.shape[0]+b_occur.shape[0]-a_in_b.shape[0])
    return jacc_index

In [58]:
def jaccard_dist(df,i,j):
    a = pd.DataFrame(df.iloc[:,i])
    b = pd.DataFrame(df.iloc[:,j])
    slice_df = pd.concat([a,b], axis=1, join="inner")
    slice_df.columns = ['A','B']
    a_in_b = slice_df[(slice_df.A>0)&(slice_df.B>0)]
    a_occur = slice_df[(slice_df.A>0)]
    b_occur = slice_df[(slice_df.B>0)]
    jacc_index = a_in_b.shape[0]/(a_occur.shape[0]+b_occur.shape[0]-a_in_b.shape[0])
    return 1-jacc_index

In [59]:
# Co-occurrence Similarity
for i in range(0,len(item_item_matrix.columns)):
    for j in range(0,len(item_item_matrix.columns)):
        item_item_matrix.iloc[i,j] = jaccard_index(join_df,i,j)

In [60]:
item_item_matrix.head()

,7-Eleven,ขวด,ขาย,ขายไม่ดี,ซีพีเมจิ,นม,รสหวาน,เบาหวาน,หาซื้อไม่ได้,กาแฟ,...,ช่องท้อง,แอนตี้ไบโอติก,ส่วนแบ่งการตลาด,แล็ป,คุมอาหาร,คนไข้,ฟื้นฟูสภาพ,อโวคาโด,เครื่องดื่มให้พลังงาน,โรคโลหิตจาง
7-Eleven,1.000000,0.023952,0.055734,0.015957,0.027144,0.017374,0.022989,0.004237,0.000000,0.008639,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ขวด,0.023952,1.000000,0.080409,0.002933,0.099312,0.083387,0.011919,0.004127,0.001458,0.034483,...,0.0,0.000000,0.001481,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ขาย,0.055734,0.080409,1.000000,0.004938,0.054435,0.091683,0.016615,0.007026,0.003686,0.049086,...,0.0,0.001244,0.002488,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ขายไม่ดี,0.015957,0.002933,0.004938,1.000000,0.003886,0.000461,0.022099,0.000000,0.000000,0.003390,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ซีพีเมจิ,0.027144,0.099312,0.054435,0.003886,1.000000,0.088480,0.021739,0.002442,0.000000,0.061616,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [61]:
item_item_matrix.to_excel(comment_cooc_jacc_file, index=True)

### 3.Create co-occurrence matrix with Association Strength สำหรับสร้างกราฟ

In [73]:
item_item_matrix = pd.DataFrame(index=join_df.columns,columns=join_df.columns).fillna(0)

In [74]:
def assoc_str(df,i,j):
    a = pd.DataFrame(df.iloc[:,i])
    b = pd.DataFrame(df.iloc[:,j])
    slice_df = pd.concat([a,b], axis=1, join="inner")
    slice_df.columns = ['A','B']
    a_in_b = slice_df[(slice_df.A>0)&(slice_df.B>0)]
    a_occur = slice_df[(slice_df.A>0)]
    b_occur = slice_df[(slice_df.B>0)]
    assocStr = a_in_b.shape[0]/(a_occur.shape[0]*b_occur.shape[0])
    return assocStr

In [75]:
# Co-occurrence Similarity
for i in range(0,len(item_item_matrix.columns)):
    for j in range(0,len(item_item_matrix.columns)):
        if i != j:
            item_item_matrix.iloc[i,j] = assoc_str(join_df,i,j)*10
        else:
            item_item_matrix.iloc[i,j] = 1

In [76]:
item_item_matrix.describe()

,7-Eleven,ขวด,ขาย,ขายไม่ดี,ซีพีเมจิ,นม,รสหวาน,เบาหวาน,หาซื้อไม่ได้,กาแฟ,...,ช่องท้อง,แอนตี้ไบโอติก,ส่วนแบ่งการตลาด,แล็ป,คุมอาหาร,คนไข้,ฟื้นฟูสภาพ,อโวคาโด,เครื่องดื่มให้พลังงาน,โรคโลหิตจาง
count,496.000000,496.000000,496.000000,496.000000,496.000000,496.000000,496.000000,496.000000,496.000000,496.000000,...,496.000000,496.000000,496.000000,496.000000,496.000000,496.000000,496.000000,496.000000,496.000000,496.000000
mean,0.003175,0.003810,0.003864,0.002539,0.003396,0.003457,0.003489,0.005636,0.003128,0.004270,...,0.018834,0.008971,0.004521,0.002304,0.002121,0.004699,0.006726,0.021037,0.007658,0.004250
std,0.045001,0.044878,0.044871,0.044998,0.044896,0.044840,0.045010,0.046562,0.045590,0.045032,...,0.145472,0.078929,0.047292,0.045094,0.044957,0.048716,0.062206,0.118316,0.072175,0.063582
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.001155,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.001349,0.001325,0.000000,0.000652,0.001456,0.000000,0.000000,0.000000,0.000630,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.001211,0.002473,0.002357,0.000000,0.001763,0.001733,0.001496,0.002574,0.000000,0.002269,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,2.500000,1.111111,1.000000,1.000000,1.000000,1.000000,1.000000,1.250000,1.111111,1.000000


In [77]:
item_item_matrix.to_excel(comment_cooc_assoc_file, index=True)

In [78]:
del item_item_matrix,join_df

## Source for Robustness Check & Cross Validation (7x7 brands matrix)

#### คำนวณ Index ของยี่ห้อต่อยี่ห้อ จากทุกๆ Attributes แสดงผลเป็น Matrix ขนาด 7x7

In [36]:
matrix_freq = pd.read_csv(root_output_path+"/cooc_bow_all.csv",index_col=0)
#cooc_matrix_freq = pd.read_excel(comment_cooc_freq_file,index_col=0)

In [37]:
brands = ['ซีพีเมจิ','ดัชมิลล์','โฟร์โมสต์','แดรี่โฮม','โชคชัย','เอ็มมิลค์','ไทยเดนมาร์ค']

In [38]:
matrix_freq_brand = matrix_freq[brands]
matrix_freq_unbrand = matrix_freq.drop(labels=brands,axis=1,inplace=False)

In [40]:
# Transpose frequency into Attribute Rating Format for precompute MDS
#cooc_matrix_freq = cooc_matrix_freq[brands]
#cooc_matrix_freq.drop(brands, axis=0, inplace=True)

In [41]:
# จำนวน Attributes = 330, brands = 7
#cooc_matrix_freq.shape

### 1.Jaccard Index

In [42]:
# REF: Lee Soojung,(2017). Improving Jaccard Index for Measuring Similarity in Collaborative Filtering
# Information Science and Applications 2017, 124.
def jaccard_index_aggregate(df_b, df_attr , i, j, k):
    brand1 = pd.DataFrame(df_b.iloc[:,i])
    brand2 = pd.DataFrame(df_b.iloc[:,j])
    attr = pd.DataFrame(df_attr.iloc[:,k])
    slice_df = pd.concat([brand1, attr, brand2], axis=1, join="inner")
    a_in_b = slice_df[(slice_df.iloc[:,0]>0)&(slice_df.iloc[:,1]>0)&(slice_df.iloc[:,2]>0)]
    a_occur = slice_df[(slice_df.iloc[:,0]>0)&(slice_df.iloc[:,1]>0)]
    b_occur = slice_df[(slice_df.iloc[:,2]>0)&(slice_df.iloc[:,1]>0)]
    jacc_index = a_in_b.shape[0]/(a_occur.shape[0]+b_occur.shape[0]-a_in_b.shape[0])
    return jacc_index

In [43]:
item_item_matrix = pd.DataFrame(index=matrix_freq_brand.columns,columns=matrix_freq_brand.columns).fillna(0)

In [44]:
jacc = pd.DataFrame(columns=matrix_freq_unbrand.columns)
jacc = jacc.append(pd.Series([],dtype='float64'), ignore_index=True).fillna(0.0)

In [45]:
# Co-occurrence Similarity
for i in range(0,len(matrix_freq_brand.columns)): # 0 to 6
    for j in range(0,len(matrix_freq_brand.columns)): # 0 to 6
        for k in range(0,len(matrix_freq_unbrand.columns)): # 0 to 329
            try:
                jacc.iloc[0,k] = jaccard_index_aggregate(matrix_freq_brand, matrix_freq_unbrand,i, j, k)
            except Exception:
                jacc.iloc[0,k] = None
                continue
        item_item_matrix.iloc[i,j] = jacc.mean(axis=1)[0]
        jacc = jacc.iloc[0:0].append(pd.Series([],dtype='float64'), ignore_index=True).fillna(0.0)

In [46]:
item_item_matrix

,ซีพีเมจิ,ดัชมิลล์,โฟร์โมสต์,แดรี่โฮม,โชคชัย,เอ็มมิลค์,ไทยเดนมาร์ค
ซีพีเมจิ,1.000000,0.153128,0.091779,0.008361,0.031330,0.061698,0.053516
ดัชมิลล์,0.153128,1.000000,0.170076,0.014292,0.063185,0.057202,0.155484
โฟร์โมสต์,0.091779,0.170076,1.000000,0.001734,0.043444,0.003081,0.139446
แดรี่โฮม,0.008361,0.014292,0.001734,1.000000,0.036736,0.012563,0.015186
โชคชัย,0.031330,0.063185,0.043444,0.036736,1.000000,0.044814,0.037228
เอ็มมิลค์,0.061698,0.057202,0.003081,0.012563,0.044814,1.000000,0.000000
ไทยเดนมาร์ค,0.053516,0.155484,0.139446,0.015186,0.037228,0.000000,1.000000


In [47]:
item_item_matrix.to_excel(comment_robust_jaccard, index=True)

### 2.Lift normalization (Netzer et al., 2011)

In [48]:
def lift_index(df,i,j):
    a = pd.DataFrame(df.iloc[:,i])
    b = pd.DataFrame(df.iloc[:,j])
    slice_df = pd.concat([a,b], axis=1, join="inner")
    slice_df.columns = ['A','B']
    p_a_in_b = slice_df[(slice_df.A>0)&(slice_df.B>0)].shape[0]/slice_df.shape[0]
    p_a_occur = slice_df[(slice_df.A>0)].shape[0]/slice_df.shape[0]
    p_b_occur = slice_df[(slice_df.B>0)].shape[0]/slice_df.shape[0]
    p_b_comprem_occur = slice_df[(slice_df.B==0)].shape[0]/slice_df.shape[0]
    lift = p_a_in_b/(p_a_occur*p_b_comprem_occur)
    return lift

In [49]:
# Follow style of Lee Soojung,(2017)
def lift_index_aggregate(df_b, df_attr , i, j, k):
    brand1 = pd.DataFrame(df_b.iloc[:,i])
    brand2 = pd.DataFrame(df_b.iloc[:,j])
    attr = pd.DataFrame(df_attr.iloc[:,k])
    slice_df = pd.concat([brand1, attr, brand2], axis=1, join="inner")
    p_a_in_b = slice_df[(slice_df.iloc[:,0]>0)&(slice_df.iloc[:,1]>0)&(slice_df.iloc[:,2]>0)].shape[0]/slice_df.shape[0]
    p_a_occur = slice_df[(slice_df.iloc[:,0]>0)&(slice_df.iloc[:,1]>0)].shape[0]/slice_df.shape[0]
    p_b_occur = slice_df[(slice_df.iloc[:,2]>0)&(slice_df.iloc[:,1]>0)].shape[0]/slice_df.shape[0]
    p_b_comprem_occur = (slice_df.shape[0]-p_b_occur)/slice_df.shape[0]
    lift = p_a_in_b/(p_a_occur*p_b_comprem_occur)
    return lift

In [50]:
#item_item_matrix = pd.DataFrame(index=cooc_matrix_freq.columns,columns=cooc_matrix_freq.columns).fillna(0)
item_item_matrix = pd.DataFrame(index=matrix_freq_brand.columns,columns=matrix_freq_brand.columns).fillna(0)

In [51]:
lift = pd.DataFrame(columns=matrix_freq_unbrand.columns)
lift = lift.append(pd.Series(), ignore_index=True).fillna(0)

In [52]:
# Co-occurrence Similarity
for i in range(0,len(matrix_freq_brand.columns)): # 0 to 6
    for j in range(0,len(matrix_freq_brand.columns)): # 0 to 6
        for k in range(0,len(matrix_freq_unbrand.columns)): # 0 to 329
            try:
                lift.iloc[0,k] = lift_index_aggregate(matrix_freq_brand, matrix_freq_unbrand,i, j, k)
            except Exception:
                lift.iloc[0,k] = None
                continue
        item_item_matrix.iloc[i,j] = lift.mean(axis=1)[0]
        lift = lift.iloc[0:0].append(pd.Series([],dtype='float64'), ignore_index=True).fillna(0.0)

In [53]:
item_item_matrix

,ซีพีเมจิ,ดัชมิลล์,โฟร์โมสต์,แดรี่โฮม,โชคชัย,เอ็มมิลค์,ไทยเดนมาร์ค
ซีพีเมจิ,1.000000,0.223833,0.146328,0.010897,0.035892,0.064705,0.103071
ดัชมิลล์,0.386693,1.000000,0.347096,0.015021,0.085453,0.063734,0.332289
โฟร์โมสต์,0.214844,0.291656,1.000000,0.001805,0.053273,0.003474,0.270999
แดรี่โฮม,0.202291,0.213137,0.050112,1.000000,0.152834,0.057719,0.257190
โชคชัย,0.268524,0.312605,0.280699,0.060430,1.000000,0.073278,0.291702
เอ็มมิลค์,0.648949,0.428774,0.037599,0.025312,0.113686,1.000000,0.000000
ไทยเดนมาร์ค,0.149538,0.249988,0.228659,0.015814,0.041234,0.000000,1.000000


In [54]:
item_item_matrix.to_excel(comment_robust_lift, index=True)

### 3. Inclusion Index (Qin He, 1999)

In [55]:
def inclu_index(df,i,j):
    a = pd.DataFrame(df.iloc[:,i])
    b = pd.DataFrame(df.iloc[:,j])
    slice_df = pd.concat([a,b], axis=1, join="inner")
    slice_df.columns = ['A','B']
    a_in_b = slice_df[(slice_df.A>0)&(slice_df.B>0)].shape[0]
    a_occur = slice_df[(slice_df.A>0)].shape[0]
    b_occur = slice_df[(slice_df.B>0)].shape[0]
    inclu = a_in_b/min(a_occur,b_occur)
    return inclu

In [56]:
def inclu_index_aggregate(df_b, df_attr , i, j, k):
    brand1 = pd.DataFrame(df_b.iloc[:,i])
    brand2 = pd.DataFrame(df_b.iloc[:,j])
    attr = pd.DataFrame(df_attr.iloc[:,k])
    slice_df = pd.concat([brand1, attr, brand2], axis=1, join="inner")
    a_in_b = slice_df[(slice_df.iloc[:,0]>0)&(slice_df.iloc[:,1]>0)&(slice_df.iloc[:,2]>0)].shape[0]
    a_occur = slice_df[(slice_df.iloc[:,0]>0)&(slice_df.iloc[:,1]>0)].shape[0]
    b_occur = slice_df[(slice_df.iloc[:,2]>0)&(slice_df.iloc[:,1]>0)].shape[0]
    inclu = a_in_b/min(a_occur,b_occur)
    return inclu

In [57]:
item_item_matrix = pd.DataFrame(index=matrix_freq_brand.columns,columns=matrix_freq_brand.columns).fillna(0)

In [58]:
inclu = pd.DataFrame(columns=matrix_freq_unbrand.columns)
inclu = inclu.append(pd.Series(), ignore_index=True).fillna(0)

In [59]:
# Co-occurrence Similarity
for i in range(0,len(matrix_freq_brand.columns)): # 0 to 6
    for j in range(0,len(matrix_freq_brand.columns)): # 0 to 6
        for k in range(0,len(matrix_freq_unbrand.columns)): # 0 to 329
            try:
                inclu.iloc[0,k] = inclu_index_aggregate(matrix_freq_brand, matrix_freq_unbrand,i, j, k)
            except Exception:
                inclu.iloc[0,k] = None
                continue
        item_item_matrix.iloc[i,j] = inclu.mean(axis=1)[0]
        inclu = inclu.iloc[0:0].append(pd.Series([],dtype='float64'), ignore_index=True).fillna(0.0)

In [60]:
item_item_matrix

,ซีพีเมจิ,ดัชมิลล์,โฟร์โมสต์,แดรี่โฮม,โชคชัย,เอ็มมิลค์,ไทยเดนมาร์ค
ซีพีเมจิ,1.000000,0.479295,0.277752,0.224371,0.288867,0.664341,0.224489
ดัชมิลล์,0.479295,1.000000,0.496723,0.224209,0.352914,0.467610,0.456192
โฟร์โมสต์,0.277752,0.496723,1.000000,0.054852,0.301146,0.040340,0.368873
แดรี่โฮม,0.224371,0.224209,0.054852,1.000000,0.219098,0.097126,0.267082
โชคชัย,0.288867,0.352914,0.301146,0.219098,1.000000,0.177778,0.300399
เอ็มมิลค์,0.664341,0.467610,0.040340,0.097126,0.177778,1.000000,0.000000
ไทยเดนมาร์ค,0.224489,0.456192,0.368873,0.267082,0.300399,0.000000,1.000000


In [61]:
item_item_matrix.to_excel(comment_robust_incur, index=True)

### 4.cosine similarity correlation matrix

In [62]:
# ต้องใช้ 1 เป็นตัวตั้งลบเสมอถ้าใช้ cosine จาก scipy.spatial.distance หากจะเอาค่า dissimliarity

def cosine_aggregate(df_b, df_attr , i, j, k):
    brand1 = pd.DataFrame(df_b.iloc[:,i])
    brand2 = pd.DataFrame(df_b.iloc[:,j])
    attr = pd.DataFrame(df_attr.iloc[:,k])
    slice_df = pd.concat([brand1, attr, brand2], axis=1, join="inner")
    a_in_b = slice_df[(slice_df.iloc[:,0]>0)&(slice_df.iloc[:,1]>0)&(slice_df.iloc[:,2]>0)]
    a_occur = a_in_b.iloc[:,0]
    b_occur = a_in_b.iloc[:,2]
    cosin = 1-cosine(a_occur,b_occur)
    return cosin

In [63]:
cosin = pd.DataFrame(columns=matrix_freq_unbrand.columns)
cosin = cosin.append(pd.Series(), ignore_index=True).fillna(0)

In [64]:
item_item_matrix = pd.DataFrame(index=matrix_freq_brand.columns,columns=matrix_freq_brand.columns).fillna(0)

In [65]:
# Similarity (1-cosine-1)
for i in range(0,len(matrix_freq_brand.columns)): # 0 to 6
    for j in range(0,len(matrix_freq_brand.columns)): # 0 to 6
        for k in range(0,len(matrix_freq_unbrand.columns)): # 0 to 329
            try:
                warnings.simplefilter("ignore", category=RuntimeWarning)
                cosin.iloc[0,k] = cosine_aggregate(matrix_freq_brand, matrix_freq_unbrand,i, j, k)
            except Exception:
                cosin.iloc[0,k] = None
                continue
        item_item_matrix.iloc[i,j] = cosin.mean(axis=1)[0]
        cosin = cosin.iloc[0:0].append(pd.Series([],dtype='float64'), ignore_index=True).fillna(0.0)

In [66]:
item_item_matrix

,ซีพีเมจิ,ดัชมิลล์,โฟร์โมสต์,แดรี่โฮม,โชคชัย,เอ็มมิลค์,ไทยเดนมาร์ค
ซีพีเมจิ,1.000000,0.895297,0.932523,0.978617,0.995961,0.997928,0.975086
ดัชมิลล์,0.895297,1.000000,0.973774,1.000000,0.995390,1.000000,0.961375
โฟร์โมสต์,0.932523,0.973774,1.000000,1.000000,0.958150,1.000000,0.965022
แดรี่โฮม,0.978617,1.000000,1.000000,1.000000,0.989649,1.000000,0.992253
โชคชัย,0.995961,0.995390,0.958150,0.989649,1.000000,0.979438,0.959926
เอ็มมิลค์,0.997928,1.000000,1.000000,1.000000,0.979438,1.000000,NaN
ไทยเดนมาร์ค,0.975086,0.961375,0.965022,0.992253,0.959926,NaN,1.000000


In [67]:
item_item_matrix.to_excel(comment_robust_cosine, index=True)

### 5.Association Strength correlation matrix

In [45]:
def assoc_str_aggregate(df_b, df_attr , i, j, k):
    brand1 = pd.DataFrame(df_b.iloc[:,i])
    brand2 = pd.DataFrame(df_b.iloc[:,j])
    attr = pd.DataFrame(df_attr.iloc[:,k])
    slice_df = pd.concat([brand1, attr, brand2], axis=1, join="inner")
    a_in_b = slice_df[(slice_df.iloc[:,0]>0)&(slice_df.iloc[:,1]>0)&(slice_df.iloc[:,2]>0)]
    a_occur = slice_df[(slice_df.iloc[:,0]>0)&(slice_df.iloc[:,1]>0)]
    b_occur = slice_df[(slice_df.iloc[:,2]>0)&(slice_df.iloc[:,1]>0)]
    assocStr = a_in_b.shape[0]/(a_occur.shape[0]*b_occur.shape[0])
    return assocStr

In [46]:
item_item_matrix = pd.DataFrame(index=matrix_freq_brand.columns,columns=matrix_freq_brand.columns).fillna(0)

In [47]:
assocStr = pd.DataFrame(columns=matrix_freq_unbrand.columns)
assocStr = assocStr.append(pd.Series(), ignore_index=True).fillna(0)

In [48]:
# Co-occurrence Similarity
for i in range(0,len(matrix_freq_brand.columns)): # 0 to 6
    for j in range(0,len(matrix_freq_brand.columns)): # 0 to 6
        for k in range(0,len(matrix_freq_unbrand.columns)): # 0 to 329
            try:
                assocStr.iloc[0,k] = assoc_str_aggregate(matrix_freq_brand, matrix_freq_unbrand,i, j, k)
            except Exception:
                assocStr.iloc[0,k] = None
                continue
        item_item_matrix.iloc[i,j] = assocStr.mean(axis=1)[0]
        assocStr = assocStr.iloc[0:0].append(pd.Series([],dtype='float64'), ignore_index=True).fillna(0.0)

In [49]:
item_item_matrix

,ซีพีเมจิ,ดัชมิลล์,โฟร์โมสต์,แดรี่โฮม,โชคชัย,เอ็มมิลค์,ไทยเดนมาร์ค
ซีพีเมจิ,0.383103,0.116387,0.077181,0.026171,0.047018,0.099373,0.054620
ดัชมิลล์,0.116387,0.473099,0.151700,0.042886,0.093746,0.074747,0.151698
โฟร์โมสต์,0.077181,0.151700,0.465034,0.005826,0.062668,0.007376,0.123044
แดรี่โฮม,0.026171,0.042886,0.005826,0.687663,0.065659,0.023275,0.051004
โชคชัย,0.047018,0.093746,0.062668,0.065659,0.563834,0.020607,0.057328
เอ็มมิลค์,0.099373,0.074747,0.007376,0.023275,0.020607,0.518409,0.000000
ไทยเดนมาร์ค,0.054620,0.151698,0.123044,0.051004,0.057328,0.000000,0.450815


In [50]:
item_item_matrix.to_excel(comment_robust_assoc, index=True)

### 5.สุ่มตัดจำนวน Row-Col เพื่อนำไปทำ QAP Test (with Jaccard Index)

สุ่ม Document 1/16, 1/8, 1/4, 1/2 เอามาทำ Cooc-matrix แล้วไล่ทำ QAP เทียบ 9000 doc กับ 4 ชุด dataset

In [71]:
np.seterr(all='ignore')
df_cooc_frq = pd.read_csv(root_output_path+"/cooc_bow_all.csv",index_col=0)
brands = ['ซีพีเมจิ','ดัชมิลล์','โฟร์โมสต์','แดรี่โฮม','โชคชัย','เอ็มมิลค์','ไทยเดนมาร์ค']

In [72]:
df_rand1 = df_cooc_frq.sample(frac=(1/16))
df_rand2 = df_cooc_frq.sample(frac=(1/8))
df_rand3 = df_cooc_frq.sample(frac=(1/4))
df_rand4 = df_cooc_frq.sample(frac=(1/2))

In [73]:
matrix_rand1_brand = df_rand1[brands]
matrix_rand1_unbrand = df_rand1.drop(labels=brands,axis=1,inplace=False)

matrix_rand2_brand = df_rand2[brands]
matrix_rand2_unbrand = df_rand2.drop(labels=brands,axis=1,inplace=False)

matrix_rand3_brand = df_rand3[brands]
matrix_rand3_unbrand = df_rand3.drop(labels=brands,axis=1,inplace=False)

matrix_rand4_brand = df_rand4[brands]
matrix_rand4_unbrand = df_rand4.drop(labels=brands,axis=1,inplace=False)

In [74]:
print(df_rand1.shape[0],df_rand2.shape[0],df_rand3.shape[0],df_rand4.shape[0])

784 1568 3136 6273


In [75]:
def jaccard_index(df_b, df_attr , i, j, k):
    brand1 = pd.DataFrame(df_b.iloc[:,i])
    brand2 = pd.DataFrame(df_b.iloc[:,j])
    attr = pd.DataFrame(df_attr.iloc[:,k])
    slice_df = pd.concat([brand1, attr, brand2], axis=1, join="inner")
    a_in_b = slice_df[(slice_df.iloc[:,0]>0)&(slice_df.iloc[:,1]>0)&(slice_df.iloc[:,2]>0)]
    a_occur = slice_df[(slice_df.iloc[:,0]>0)&(slice_df.iloc[:,1]>0)]
    b_occur = slice_df[(slice_df.iloc[:,2]>0)&(slice_df.iloc[:,1]>0)]
    jacc_index = a_in_b.shape[0]/(a_occur.shape[0]+b_occur.shape[0]-a_in_b.shape[0])
    return jacc_index

In [76]:
# Co-occurrence Similarity
def create_Random_Poportion(df_b, df_attr):
    precomputed_matrix = pd.DataFrame(index=df_b.columns,columns=df_b.columns).fillna(0)
    
    jacc = pd.DataFrame(columns=df_attr.columns)
    jacc = jacc.append(pd.Series([],dtype='float64'), ignore_index=True).fillna(0.0)
    
    for i in range(0,len(precomputed_matrix.columns)):
        for j in range(0,len(precomputed_matrix.columns)):
            for k in range(0,len(df_attr.columns)): # 0 to 329
                try:
                    jacc.iloc[0,k] = jaccard_index(df_b, df_attr,i, j, k)
                except Exception:
                    jacc.iloc[0,k] = None
                    continue
            precomputed_matrix.iloc[i,j] = jacc.mean(axis=1)[0]
            jacc = jacc.iloc[0:0].append(pd.Series([],dtype='float64'), ignore_index=True).fillna(0.0)
    return precomputed_matrix

In [77]:
item_item_matrix = create_Random_Poportion(matrix_rand1_brand,matrix_rand1_unbrand)
item_item_matrix.to_excel(root_output_path+"/poportion1.xlsx", index=True)

In [78]:
item_item_matrix = create_Random_Poportion(matrix_rand2_brand,matrix_rand2_unbrand)
item_item_matrix.to_excel(root_output_path+"/poportion2.xlsx", index=True)

In [79]:
item_item_matrix = create_Random_Poportion(matrix_rand3_brand,matrix_rand3_unbrand)
item_item_matrix.to_excel(root_output_path+"/poportion3.xlsx", index=True)

In [80]:
item_item_matrix = create_Random_Poportion(matrix_rand4_brand,matrix_rand4_unbrand)
item_item_matrix.to_excel(root_output_path+"/poportion4.xlsx", index=True)

#### NOTE
การเทียบ QAP - matrix จะต้องเป็น sim หรือ dissim เหมือนกันทั้งสองฝั่ง (ในที่นี้ใช้ sim)<br>
การออกกราฟ - matrix ใช้ Similarity<br>
การทำ MDS - matrix ใช้ Dissimilarity (distance) เท่านั้น<br>